In [23]:
import time
import pandas as pd
from py2neo import Graph, Node, Relationship

In [24]:
def query_to_df(query, graph):
    print("Starting query...", end=" ")
    query_start_time = time.time()
    df = graph.run(query).to_data_frame()
    print("Done ({:.2f} minutes).".format((time.time()-query_start_time)/60))
    return df

In [25]:
graph = Graph( "bolt://matlaber5.media.mit.edu:7687", auth=('neo4j','myneo'))
print("Connected to graph database with {:,} nodes and {:,} relationships!".format(
    graph.database.primitive_counts['NumberOfNodeIdsInUse'], 
    graph.database.primitive_counts['NumberOfRelationshipIdsInUse']))

Connected to graph database with 278,432,359 nodes and 1,844,501,832 relationships!


In [4]:
top_5 = ['Cell', 'Nature', 'Nature Biotechnology','Proceedings of the National Academy of Sciences of the United States of America','Science']
top_10 = ['Cell', 'Nature', 'Nature Biotechnology','Proceedings of the National Academy of Sciences of the United States of America','Science', 'Journal of the American Chemical Society', 'JAMA', 'The New England Journal of Medicine', 'Nature Genetics', 'Neuron']
top_42 = ['Angewandte Chemie','Blood','Cancer Cell','Cancer Discovery','Cancer Research','Cell','Cell Host & Microbe','Cell Metabolism','Cell Stem Cell','Chemistry & Biology','The EMBO Journal','Genes & Development','Immunity','Journal of Neurology','Journal of the American Chemical Society','JAMA','Journal of Biological Chemistry','Journal of Cell Biology','Journal of Clinical Investigation','Journal of Experimental Medicine','Journal of Medicinal Chemistry','The Lancet','Nature Cell Biology','Nature Chemical Biology','Nature Chemistry','Nature Medicine','Nature Methods','Nature','Nature Biotechnology','The New England Journal of Medicine','Neuron','Nature Genetics','Nature Immunology','Nature Neuroscience','Nature Structural & Molecular Biology','PLOS Biology','PLOS Genetics','PLOS Pathogens','Proceedings of the National Academy of Sciences of the United States of America','Science Signaling','Science Translational Medicine','Science']

In [3]:
import pandas as pd
df_impact_rank = pd.read_csv('~/Workspace/UROP/scaling-science/notebooks/Collaboration/Data/impact_rank_1000.csv',encoding = "UTF-8")
top_profs = set()
for prof in df_impact_rank['avg_PR']:
    top_profs.add(prof)
for prof in df_impact_rank['avg_AR']:
    top_profs.add(prof)
top_profs = list(top_profs)
print(len(top_profs))

1208


In [17]:
f =  open('/home/brendan/Workspace/UROP/scaling-science/notebooks/Collaboration/Data/impact_1000.txt', 'w')
f.write('[')
for prof in top_profs[:-1]:
    f.write("'" + prof + "', ")
f.write("'" + top_profs[-1] + "']")
f.close()

In [21]:
['David P. Reed', 'Mikulas Popovic', 'Katherine M. Flegal', 'Boris Reva', 'Emily S. Winn-Deen', 'B. G. Barrell', 'Richard G. Margolese', 'John M. Chirgwin', 'Floyd Ratliff', 'Joan Massagué', 'Sarah Milne', 'M. Overgaard', 'Joseph L. Goldstein', 'Todd M. Lowe', 'Jonathan Marchini', 'Peter J. Ramadge', 'Daniel R. Marshak', 'Stephen M. J. Searle', 'Leon Festinger', 'James Kennedy', 'Michael B. Mock', 'Juha Kononen', 'Julia B. Lewis', 'Jonathan R. Edwards', 'Victor E. Velculescu', 'Peter J. Turnbaugh', 'Akira Nagafuchi', 'Amy E. Pasquinelli', 'Norman Arnheim', 'Thomas D. Schmittgen', 'Linda M. Rolnitzky', 'Rodrigo Lopez', 'J.-C. Gazet', 'Joel S. Hayflick', 'P. C. Eklund', 'Stefanie S. Jeffrey', 'Camille Parmesan', 'N. Cabrera', 'Andrew J. Mungall', 'Robert Baertsch', 'Robert E. Bruccoleri', 'Robert S. Pinals', 'Helen Skaletsky', 'Keith Nelson', 'J. Richard Hackman', 'J. Michael Cherry', 'Mark F. Pittenger', 'Richa Agarwala', 'Mark A. DePristo', 'James H. Ware', 'Serafim Batzoglou', 'Lee A. Green', 'J. E. Till', 'Elizabeth Ward', 'Sheila Scully', 'William T. Friedewald', 'Allan M. Skene', 'V. Narry Kim', 'Tina Graves', 'Elaine R. Mardis', 'George W. Torrance', 'Robert L. Coffman', 'Andreas Heger', 'Ellen E. Walters', 'Jeremy D. Selengut', 'Ruth E. Ley', 'C. V. Shank', 'Gregory J. Hannon', 'Donald S. Rae', 'James A. Stockman', 'Saira Salahuddin', 'J. G. Bednorz', 'D. von Fournier', 'J. F. A. P. Miller', 'Scott Schwartz', 'Ivica Letunic', 'Heike Fiegler', 'J. Gillis', 'Russell Ross', 'Christie Kovar', 'Bruce W. Birren', 'Richard S. Lazarus', 'Helga Boedtker', 'George A. Miller', 'George H. Mead', 'Arthur Purdy Stout', 'LaDeana W. Hillier', 'William A. Sibley', 'Michael P. Bevilacqua', 'Michele L. Pearson', 'Jean-Roger Le Gall', 'M. R. Cooper', 'Jean Thierry-Mieg', 'Joanna E. Siegel', 'Raffaella Sordella', 'David J. Spiegelhalter', 'Frans Bruyninckx', 'Bruce M. Spiegelman', 'George G. J. M. Kuiper', 'Stacey Gabriel', 'Seymour S. Kety', 'Kenneth F. Schulz', 'S. L. McCall', 'Morris W. Foster', 'Huaiyu Mi', 'Nick Patterson', 'Donna K. Slonim', 'Richard H. Chapman', 'Clare Seamark', 'Saul A. Teukolsky', 'Craig C. Mello', 'E Robinson', 'A E Weyman', 'Hartmut Land', 'Ramon Parsons', 'Katsuaki Hoshino', 'George J. Taylor', 'Deborah Wentworth', 'M Richard', 'Thomas E. Carew', 'David J. Lipman', 'Erich Bluhmki', 'John D. McPherson', 'Robert J. Garrison', 'William F. Novotny', 'Kary B. Mullis', 'Robert M. Weisskoff', 'Zemin Ning', 'Daniel Shoemaker', 'M Bond', 'Jillian Boreham', 'Zoltán N. Oltvai', 'Jay R. Ritter', 'Leland H. Hartwell', 'G. Binnig', 'Tomaso Poggio', 'Nathan Maccoby', 'Donald D. Newmeyer', 'Benjamin A. Barnes', 'Charles Tanford', 'M. Michele Manos', 'Otto A. Bessey', 'Yudong D. He', 'Herbert A. Simon', 'Deborah Estrin', 'John V. Moran', 'Gabor Gyapay', 'Michael Shi', 'Edward J. Roccella', 'Richard D. Piner', 'Taro Kawai', 'Erica Sodergren', 'Stephen J. Aldington', 'M. Y. Kim', 'Gilean McVean', 'John P. Perdew', 'Scott Shenker', 'S. Moncada', 'Anders Krogh', 'Jerome Cornfield', 'Ralph M. Steinman', 'M. E. J. Newman', 'Jeffrey C. Lee', 'Benjamin D. Hall', 'Miriam J. Alter', 'Sidney Udenfriend', 'James K. V. Willson', 'Mark S. Guyer', 'Julian Peto', 'James E. Galagan', 'Alice C. Young', 'Theophil Staehelin', 'Steven D. Averbuch', 'Edmund G. Lowrie', 'Cliff Tabin', 'P Sandercock', 'Nathan Goodman', 'Richard A. Shafer', 'Steve Rozen', 'A. R. Hunt', 'J.A. van Dongen', 'Mary C. Betlach', 'Michael C. Zody', 'Harry Towbin', 'Myunghee C. Paik', 'Mark Diekhans', 'Michael Gent', 'Mark S. Boguski', 'Hongjie Dai', 'David V. Goeddel', 'Andrew Neil', 'Yunzhen Cao', 'M. H. Gault', 'Paul Meier', 'Robert E. Kleiger', 'Kim Rutherford', 'William R. Pearson', 'Janan T. Eppig', 'Bert Sakmann', 'Stuart McLaren', 'R Parker', 'James H. Jandl', 'David P. Bartel', 'C Redmond', 'Jerome Vinograd', 'Mark Dickson', 'Barry A. Morgan', 'Julie D. Thompson', 'Mike Rothe', 'Tobias Doerks', 'Jeffrey G. Reid', 'D. J. E. Ingram', 'Yuling Hong', 'Barbara J. McNeil', 'Robert Tibshirani', 'Jeremy H. Burroughes', 'Ueli Gubler', 'C. Milstein', 'Paul J. Flory', 'A. K. Mitchell', 'Mark T. Ross', 'I. M. Stratton', 'P. D. Home', 'Andrew P. Holmes', 'C. Arden Pope', 'Tim R. Mosmann', 'Richard Peto', 'Richard C. Graham', 'John M. Leonard', 'Arthur B. Lamb', 'Benoit B. Mandelbrot', 'David E. Goldberg', 'E. Kellenberger', 'F. Schedin', 'C. K. Osborne', 'Nicholas Kassebaum', 'Ch. Gerber', 'Russell Turner', 'Stéphane Guindon', 'Frank E. Speizer', 'Susan G. Arbuck', 'Thomas D. Willis', 'R W Davis', 'Phillip D. Zamore', 'Nan M. Laird', 'Barth Hoogstraten', 'Kim C. Worley', 'Hugo Muench', 'Douglas Hanahan', 'Margaret D. Carroll', 'Gary Ruvkun', 'B. E. Warren', 'Catherine A. Lozupone', 'Robert F. Meenan', 'Naruya Saitou', 'James S. Marks', 'Wilson Ia', 'Minze Stuiver', 'Seymour Benzer', 'Terrence S. Furey', 'Judah Folkman', 'Richard E. Nisbett', 'Pui Yan Kwok', 'Mark A. Saper', 'Erika Schmid', 'Marita Cross', 'Dennis G. Ballinger', 'Elisabeth Burdick', 'D. Wayne Taylor', 'Michele Cargill', 'Fred Ederer', 'Loren F. Hiratzka', 'Oliver Schwardt', 'Saul Genuth', 'Stuart Bunting', 'Arnold Oliphant', 'N. E. Day', 'Paul Predki', 'Paul D. Sorlie', 'Donna M. Rennick', 'Louis A. Tartaglia', 'Parnesh Raniga', 'Robert Axelrod', 'A. McGuire', 'Stewart Scherer', 'Lucian L. Leape', 'U. J. Lewis', 'J Burns', 'Sarah E. Fowler', 'Bernard B. Brodie', 'Ghia Euskirchen', 'Kate Downes', 'Scott M. Hammond', 'Robert D. Fleischmann', 'Robert M. Kuhn', 'Richard J. Brand', 'Jerome Y. Lettvin', 'Thomas R. Fleming', 'Joseph D. Mosca', 'Erin Pleasance', 'Nahida H. Gordon', 'Calvin B. Harley', 'Russell Higuchi', 'Réka Albert', 'Keaven M. Anderson', 'J. F. Forbes', 'M Fernandez', 'Zhiping Gu', 'Bala Swaminathan', 'Jeffrey Vieira', 'Annie Yang', 'P. B. Hirsch', 'R. S. J. Frackowiak', 'Tavia Gordon', 'Ralf W. Grosse-Kunstleve', 'James R. Rice', 'V. S. Kalyanaraman', 'Daniel C. Koboldt', 'James B. Hicks', 'Jerome S. Bruner', 'Elia Stupka', 'Michael C. Jensen', 'Randi Y. Leavitt', 'Stephen F. Altschul', 'Hidemi Watanabe', 'Andrey Sivachenko', 'William J. Rutter', 'Michael Fiorino', 'Louis Fehrenbacher', 'James J. Gibson', 'Béatrice Vanbervliet', 'Richard C. Eastman', 'Qui-Lim Choo', 'Rodger Staden', 'Jeffrey C. Barrett', 'Donald B. Rubin', 'Fredrik Ronquist', 'Fuli Yu', 'Reubin Andres', 'Jacob Cohen', 'Joseph H. Roe', 'Troy Moore', 'David Tilman', 'Zhongwu Lai', 'Jonathan Sebat', 'Wonhee Jang', 'David W. Bilheimer', 'P. Armitage', 'Kalevi Pyörälä', 'John E. Ware', 'J. J. Hopfield', 'Norman E. Breslow', 'Julius Marmur', 'J. D. A. Van Embden', 'Christoph Kalka', 'Eric S. Lander', 'George L. Gabor Miklos', 'Douglas B. Rusch', 'Katherine S. Elliott', 'Alan Coulson', 'Jeffrey N. Rottman', 'Scott V. Tingey', 'Arne Tiselius', 'Dennis A. Gilbert', 'Wendy Winckler', 'Sean Humphray', 'Max Halperin', 'E. Wolf', 'Douglas L. Johnson', 'R. C. Turner', 'Arthur M. Sobol', 'Cladd E. Stevens', 'S. Cobbe', 'E. P. Greenberg', 'Howard H. Hiatt', 'P. J. Randle', 'E. Orowan', 'Aaron T. Beck', 'M Klagsbrun', 'Yves-Alain Barde', 'Daniel Kahneman', 'Robert T. DeBoy', 'G. E. Palade', 'Adam Frankish', 'Sam T. Roweis', 'Andersen Kw', 'Thomas J. Dull', 'Michaele C. Christian', 'David R. Cox', 'Donald Geman', 'M. H. F. Wilkins', 'William J. Henzel', 'Brygg Ullmer', 'Carol Redmond', 'Michael B. Eisen', 'Chang Gong Liu', 'Paul Milgrom', 'Trampisch Hj', 'M. D. Poulik', 'Daniel W. Bradley', 'Carl-Magnus Rudenstam', 'Albert László Barabási', 'David S. Hogness', 'Michael L. Metzker', 'Donald Pearson', 'J. S. Turner', 'James F. Fries', 'Suzannah Bumpstead', 'Ben Z. Locke', 'Marianne Kearney', 'R. Mead', 'Tim Hubbard', 'William Saurin', 'R. R. Ernst', 'Federico Girosi', 'Peter J. Sabo', 'Alex Bateman', 'Dienstag Jl', 'Ed Harlow', 'Carrie Sougnez', 'Guy Plunkett', 'Richard Bellman', 'Aaron McKenna', 'Gary J. Olsen', 'Lawrence L. Kupper', 'Mark D. Adams', 'Luther T. Clark', 'Janet M. Corrigan', 'A. F. Huxley', 'Nigel P. Carter', 'D Wright', 'William H. Stein', 'Robert S. Fulton', 'Heidi Greulich', 'Sankar Swaminathan', 'Toby J. Gibson', 'Simon G. Gregory', 'Aravind Subramanian', 'C. Katz', 'Tetsuo Maita', 'Helena Hämäläinen', 'Eugene W. Myers', 'Rhian Gwilliam', 'Anup Madan', 'Forrest E. Kendall', 'Cynthia L. Ogden', 'Aileen Robertson', 'Harold T. Dodge', 'Marilyn Bergner', 'D. R. Cox', 'G. I. Taylor', 'Jonathan Butler', 'Andrew B. Lassar', 'S Mason', 'Roland M. H. Schein', 'Margo A. Denke', 'Richard M. Goldstein', 'Robert B. Catalano', 'Alexander E. Urban', 'Harvey J. Alter', 'Monique G. Lê', 'Gyorgy Hutvagner', 'Darren A. Natale', 'Stanley J. Korsmeyer', 'J. Craig Venter', 'D Nelson', 'Richard Tizard', 'W Jack', 'Lee Rowen', 'Hailing Hsu', 'Robert P. Gaynes', 'Donald R. Huffman', 'Richard H. Creech', 'Michael H. Kutner', 'Michael A. Saunders', 'R Peto', 'Carol J. Bult', 'Ian Korf', 'D. R. Curtis', 'Michael Held', 'Michael R. Hee', 'Ali A. Mokdad', 'Frank Delaglio', 'Donald S. Fredrickson', 'Jeffrey S. Beck', 'Shirley Zhu', 'Jorge J. Yunis', 'Richard A. Olshen', 'David Altshuler', 'Gabriele Bergers', 'Sandra W. Clifton', 'Colin N. Dewey', 'Georges Köhler', 'Paul T. Spellman', 'Stanley Lemeshow', 'Dominic A. Scudiero', 'Basil M. Rifkind', 'Walter M. Fitch', 'Endel Tulving', 'J. M. Bland', 'Nikola P. Pavletich', 'Nawab Qizilbash', 'Raymond J. MacDonald', 'W. H. Zachariasen', 'Daniel T. Colbert', 'D. R. Matthews', 'Patrick Brill-Edwards', 'Michael K. Showe', 'Marc van de Wetering', 'Bela Julesz', 'Earl Hubbell', 'Scott L. Zeger', 'Peter K. Wellauer', 'Zbyszek Otwinowski', 'Charles T. Kresge', 'Helen Van Vunakis', 'Nicoletta Sacchi', 'Douglas G. Altman', 'R Doll', 'Richard W. Reynolds', 'A. Klug', 'Amos Tversky', 'Quinn L. Deveraux', 'Patrick O. Brown', 'Patrick D. Wall', 'David Botstein', 'K Matsushima', 'Brufman G', 'Zhaodan Cao', 'Elisabeth Buchdunger', 'Alex Pentland', 'F. P. Bowden', 'J Holmes', 'Kenneth J. Livak', 'K Sakai', 'Hoda Khouri', 'Jerzy Neyman', 'R. Willox', 'Raymond G. Goodwin', 'Richard Durbin', 'Alan J. Buckler', 'Fu Lu', 'Gordon J', 'Pablo Tamayo', 'Shimon Ullman', 'Gary W. Trucks', 'Morris J. Karnovsky', 'J. S. Waugh', 'Wen Lin Kuo', 'Nicholas B. Lydon', 'Amy J. Weiner', 'Anne S. Olsen', 'Allan C. Wilson', 'Daniel H. Haft', 'Stephen E. Haynesworth', 'Marek Kubin', 'Carol Churcher', 'Todd D. Taylor', 'Jane Rogers', 'Teresa C. Horan', 'S. Korzeniowski', 'Nick Goldman', 'John P. Huelsenbeck', 'Huyen Dinh', 'Maynard V. Olson', 'Vern Paxson', 'Joel E. Richardson', 'John C. Cassel', 'Josep F. Abril', 'Robert F. Kibler', 'H. Gervásio', 'David I. Gottlieb', 'Sarah Edkins', 'Howard K. Schachman', 'David Beach', 'Andrew P. Feinberg', 'Hilary King', 'John C. Doyle', 'Jon Teague', 'Laurence Kennedy', 'Anthony R. Kerlavage', 'Drummond Rennie', 'Allyson Ross Davies', 'George Kuo', 'Jian Bing Fan', 'Gwyneth Van', 'Paul McGale', 'Ellson Y. Chen', 'Trevor Hastie', 'Pierre R. Villeneuve', 'Thomas Tuschl', 'Richard R. Copley', 'Torben Palshof', 'James R. Hudson', 'Sandra Silberman', 'John McCarthy', 'Wieslaw J. Roth', 'Douglas J. Schwartzentruber', 'Ian Dunham', 'Carol W. Greider', 'Leslie L. Iversen', 'Sean R. Eddy', 'Claudia A. Seipp', 'Frederic M. Richards', 'Lawrence G. Hunsicker', 'Rajinder Kaul', 'Mark R. Chassin', 'Susan E. Folstein', 'Richard Breathnach', 'Thomas R. Dawber', 'E. A. McCulloch', 'Stephen C. Reingold', 'Mukesh Dherani', 'F Taylor', 'Santos A. Susin', 'Simon Myers', 'Ronald H. Reeder', 'Hilary A. Coller', 'James E. Grizzle', 'Lynn A. G. Ries', 'Eugene L. Brown', 'Robert Karasek', 'Edward Witten', 'Myron Flickner', 'Oliver H. Lowry', 'A. L. Hodgkin', 'Kenneth J. Arrow', 'Jorja G. Henikoff', 'G. E. Hutchinson', 'Nancy A. Thornberry', 'Naoufal Zamzami', 'A. P. Alivisatos', 'Vincent P. Stanton', 'E Thomson', 'Deanna M. Church', 'Mark S. Chee', 'Joe M. McCord', 'François Artiguenave', 'Liisa Holm', 'Eladio Viñuela', 'Mary Burden', 'Granger G. Sutton', 'Oliver Smithies', 'Herbert W. Boyer', 'William H. Habig', 'J Graham', 'J. R. Vane', 'Therese Sørlie', 'Audrey Duncanson', 'Marjorie B. Lees', 'Franklin Kim', 'Maurizio Belfiglio', 'C Davies', 'Edmund A. Gehan', 'Jill P. Mesirov', 'Alastair Kent', 'John R. Glover', 'Susan M. Hailpern', 'Patrick Therasse', 'Sarah Hunt', 'Arthur L. Delcher', 'Gavin Sherlock', 'Martin Gibson', 'Robert P. Abelson', 'Kristin Ardlie', 'Timothy B. Stockwell', 'Zhengdong D. Zhang', 'Stewart Craig', 'Charles Dauguet', 'Thomas Jenuwein', 'Dean Burk', 'Joseph Ragaz', 'Renaud Capdeville', 'Halsted R. Holman', 'Mark H. Skolnick', 'Harold Swerdlow', 'Jennifer Baldwin', 'Roger E. Kirk', 'Erik L. L. Sonnhammer', 'Adam Auton', 'Abraham Kagan', 'Marianne Sullivan', 'Torsten N. Wiesel', 'Vishwanath R. Iyer', 'W. Brian Arthur', 'Michael L. Watson', 'Darrel A. Regier', 'Graeme A. Reid', 'D. H. Auston', 'Srinivasa M. Srinivasula', 'Daniel I. Arnon', 'J Clarke', 'Marshal F. Folstein', 'Tullia Lindsten', 'Robert J. Lipshutz', 'Lucy Matthews', 'Edwin M. Southern', 'William B. Kannel', 'James C. Mullikin', 'Matthew Stephens', 'Jane Grimwood', 'Ross L. Prentice', 'Laura Elnitski', 'Alexandre Montpetit', 'M. G. Bawendi', 'Nathan O. Kaplan', 'Jim Gray', 'Elizabeth Lacy', 'Mark Yandell', 'Christopher B. Burge', 'Andrew Kasarskis', 'Alan T. Bankier', 'Clifford L. Johnson', 'Koichiro Tamura', 'Harold Weintraub', 'L. Di Martino', 'Robert A. Weinberg', 'Susan L. Naylor', 'Andrew Wakeham', 'Karin Eiglmeier', 'John G. Kirkwood', 'Scott Uknes', 'Steven E. Scherer', 'Jacques Ferlay', 'David L. Donoho', 'Timothy Fennell', 'Genji Matsuda', 'Barbara E. Stranger', 'Eric A. Jaffe', 'Daphne W. Bell', 'D. Williams Parsons', 'John V. Bennett', 'Matt van de Rijn', 'Parkhurst A. Shore', 'Paul Liska', 'Aram V. Chobanian', 'James J. Heckman', 'Simon L. Howell', 'R. W. Butcher', 'S. M. Zahurak', 'Pamela Whittaker', 'Robert A. Rosati', 'Sarah Nutland', 'David Torrents', 'Yoav Freund', 'Gabor T. Marth', 'Catherine A. Ball', 'T. N. Bhat', 'Mark J. Daly', 'Eric A. Swanson', 'Scott L. Weinrich', 'Robert Katzman', 'Taylor A. Sohn', 'Erding Hu', 'Andreas Thess', 'Sj Wilson', 'Ants Leetmaa', 'P. F. Scholander', 'Allen Newell', 'Argiris Efstratiadis', 'John P. Feighner', 'Lisa French', 'Larry Rubinstein', 'A. L. Page', 'Catherine Michaud', 'Jean Endicott', 'Jacques Monod', 'Anne Dornhorst', 'Kara Dolinski', 'Stanley L. Wallace', 'Kurt J. Greenlund', 'Emily Bernstein', 'Francine Brière', 'Ronald Melzack', 'Gerald M. Rubin', 'Richard Helm', 'C Fox', 'Robert G. Parr', 'David B. Johnson', 'Lon R. Cardon', 'Gail R. Martin', 'David L. Swofford', 'Roland Heilig', 'Helen Goodell', 'John H. Holland', 'Francis M. Miezin', 'Frederick Mosteller', 'J.Ward Kennedy', 'Sidney J. Cutler', 'Michael S. Gottlieb', 'John R. Marler', 'Keith J. Worsley', 'Waldo E. Cohn', 'Joan Keesey', 'Andrew Y. Ng', 'Brian Haynes', 'U. Veronesi', 'Warren J. Hehre', 'Allan Van Oosterom', 'Martin Fishbein', 'Arian Smit', 'James B. Wyngaarden', 'Robin Humphry-Baker', 'David G. Lowe', 'Charles D. Keeling', 'Alan D. Lopez', 'Kimmen Sjölander', 'David G. Kleinbaum', 'W. James Kent', 'J Martin Bland', 'Robert J. Dodson', 'K. S. Novoselov', 'Mercedes Camalli', 'Alexej Abyzov', 'Melvin P. Judkins', 'William H. Majoros', 'D. Lawrence Wickerham', 'Paul Doty', 'Ewan Birney', 'Daniel W. Jones', 'Kung-Yee Liang', 'Gordon Saksena', 'Solange Chamaret', 'Frederick C. Neidhardt', 'John Nash', 'Michael S. Waterman', 'Thomas N. Sato', 'M. Castiglione', 'Geoffrey E. Hinton', 'Wolfgang Kabsch', 'David Julius', 'David L. Pearle', 'Dennis Y. Loh', 'Ronald A. Laskey', 'Thomas C. Chalmers', 'Noriko Sakaguchi', 'Eric Pelletier', 'Jeffrey A. Cutler', 'Thomas J. Palker', 'Michael D. McLellan', 'M. G. Sarngadharan', 'Michael Feolo', 'Taylor Ja', 'Valerie A. Fadok', 'Andreas Rump', 'F. C. Greenwood', 'Vladimir Korinek', 'Reto Ruedy', 'Lore Zech', 'A. C. T. North', 'Peter W. J. Rigby', 'Donald P. Nierlich', 'Edgardo Lander', 'Jeremy D. Glasner', 'Sarah S. Murray', 'Rebecca DerSimonian', 'Clifton F. Mountain', 'David R. Bentley', 'R. A. Fisher', 'Andrew Fire', 'Gordon L. Shulman', 'Anita L. Stewart', 'Ahmedin Jemal', 'Desmond G. Higgins', 'John M. Darley', 'H. J. Stewart', 'Isaac L. Chuang', 'David J. Dooling', 'David W. Tank', 'Yuval Eshed', 'Monica Williams', 'Kikuko Hotta', 'Nj Bell', 'Kazuto Kato', 'Linus Pauling', 'Freddie Bray', 'Bert Vogelstein', 'K McPherson', 'Werner Risau', 'Evan Mauceli', 'Federica Sallusto', 'Charles Atkinson', 'Paul Emsley', 'D. C. Van Essen', 'P Park', 'John D. Lafferty', 'Donna F. Stroup', 'Mark Weiser', 'Charles H. Hennekens', 'Beverley Shields', 'Hugh Paterson', 'Ken Dewar', 'John E. Helzer', 'Sanford L. Palay', 'George Paxinos', 'Liesi E. Hebert', 'Robert C. Onofrio', 'Rachel M. Freathy', 'David A. Hungerford', 'Yasunori Ogura', 'Bruce N. Ames', 'Sam Griffiths-Jones', 'Catrina Fronick', 'Ivan Lieberburg', 'Stephen T. Sherry', 'Charmaine D. Royal', 'N. F. Mott', 'Douglas T. Ross', 'H. Murayama', 'Marshall N. Rosenbluth', 'Rodney M. Hewick', 'W. M. Cowan', 'Karen Hsiao', 'Timo T. Valle', 'Yasushi Totoki', 'Amos Marc Bairoch', 'William A. Knaus', 'William J. Fulkerson', 'Christopher K. Raymond', 'Ross A. Lippert', 'John A. Pople', 'W. Richard McCombie', 'Lora Lewis', 'Michael W. Hunkapiller', 'B. Gutenberg', 'Karen Eilbeck', 'John R. Carson', 'Jeremy White', 'Shelley E. Taylor', 'Tarjei S. Mikkelsen', 'Vincent Ferretti', 'Thomas H. Holmes', 'Karsten Hokamp', 'David Haussler', 'Graham R. Bignell', 'W. Dansgaard', 'Paul Ekman', 'Amy A. Caudy', 'Ingrid Daubechies', 'B. A. Roe', 'A. Howie', 'Joachim Messing', 'Franz Josef Och', 'Eli Robins', 'Bates Dw', 'H. R. Horvitz', 'Michael Ashburner', 'Hamilton O. Smith', 'William N. Anderson', 'Kenneth W. Kinzler', 'Joseph H. Connell', 'Tom Maniatis', 'John M.C. Gutteridge', 'Suzanna E. Lewis', 'Rolf Apweiler', 'Robert I. Levy', 'Paul J. Deutsch', 'Eric W. Wong', 'Robin Milner', 'Keith R. Porter', 'Stefan Taudien', 'S Takashima', 'Michael J. Thun', 'Joe Sambrook', 'E. A. Clarke', 'David W. Leung', 'Marco Punta', 'M. Karplus', 'B. F. Skinner', 'Vishva M. Dixit', 'Andrew G. Fraser', 'Brian R. Lawn', 'Max H. Myers', 'Sewall Wright', 'Martine Van Glabbeke', 'Bruce Culleton', 'Martin L. Hooper', 'Nancy F. Hansen', 'Albert J. Belanger', 'Kirk R. Thomas', 'Andrew Menzies', 'Stanford Moore', 'Miller Ja', 'Max Born', 'Ulrich K. Laemmli', 'Walter Gilbert', 'Tracy M. Bryan', 'G. Flandrin', 'Alfonse T. Masi', 'Giorgio Cocconi', 'James H. Stock', 'Nathan Mantel', 'Kenneth P. Burnham', 'Lawrence E. Shulman', 'Sheila Seal', 'David S. Johnson', 'Gonçalo R. Abecasis', 'Donald E. Groom', 'Ramzi S. Cotran', 'S Clarke', 'Owen P. Hamill', 'A. K. Geim', 'William P. Castelli', 'Jinghui Zhang', 'Folke Skoog', 'Alejandro A. Schäffer', 'George E. Palade', 'Paul E. Meehl', 'Lee Murphy', 'Antman Em', 'René O. Mirimanoff', 'Bonnee Rubinfeld', 'Jennifer J. Anderson', 'Gitanjali M. Singh', 'Anthony J. Cox', 'Richard O. Russell', 'David L. Sackett', 'Felsenfeld Al', 'Jane Peterson', 'H. Craig', 'M Schumacher', 'Sergey V. Morozov', 'Jacqueline Kosecoff', 'P. J. Flory', 'Irving Langmuir', 'Albert H. Coons', 'Roel P. Funke', 'Steven L. Salzberg', 'Alane Gray', 'James W. Curran', 'Lars Hedin', 'G. A. R. Johnston', 'G. Bonadonna', 'Auke Tellegen', 'Michael R. Brent', 'al. et', 'Albert-László Barabási', 'Earl W. Sutherland', 'William Haenszel', 'Dmitriy A. Dikin', 'J. P. Grime', 'N. Ferrara', 'A. D. Bangham', 'Karen A. Ketchum', 'Sidney C. Smith', 'Douglas P. Wagner', 'Don W. Fawcett', 'Steven H. Strogatz', 'Jonathan K. Pritchard', 'Toby Bloom', 'Deborah R. Shapiro', 'Phyllis A. Wingo', 'Ronald L. Rivest', 'I. V. Grigorieva', 'Mary Osborn', 'Glenn Tesler', 'P. J. Sellers', 'F. Sanger', 'David H. Gelfand', 'Howard A. Eder', 'J. L. Weiland', 'Jennifer R. Wortman', 'Daniel McGee', 'G. D. Farquhar', 'John F. Heidelberg', 'Yuen Shing', 'Brenda K. Edwards', 'John Bowlby', 'Ruslan Medzhitov', 'Ching W. Tang', 'Norman A. Doggett', 'Gerald R. Fink', 'Trevor Hawkins', 'Samuel B. Guze', 'Richard R. Hudson', 'Solomon A. Berson', 'Eugene R. Passamani', 'David B. Jaffe', 'Roger Y. Tsien', 'Bigger Jt', 'Victor R. Ambros', 'David H. Culver', 'Donald D. Van Slyke', 'John E. Wennberg', 'McGuire Wl', 'Wayne J. Higgins', 'Jingtao Sun', 'Reginald A. Clift', 'Jitendra Malik', 'Gordon G. Wong', 'Fred Ramsdell', 'Heather Schmidt', 'J. S. Bell', 'P. G. Smith', 'Michael L. Frigge', 'Gilbert Chu', 'Irene M. Stratton', 'Martin Ringwald', 'Katherine W. Timothy', 'William H. Dietz', 'Joseph L. Fleiss', 'E Howard', 'Yuri Lazebnik', 'Cyrus Levinthal', 'Stephen Frank Schaffner', 'M. A. Fiatarone', 'Cole Trapnell', 'J. Milburn Jessup', 'Chris C A Spencer', 'Richard M. Karp', 'Henry A. Erlich', 'Jörg Schultz', 'Gary Struhl', 'Steven R. Gill', 'Patrick Minx', 'Craig H. Mermel', 'L. J. Ignarro', 'Kevin McKernan', 'Ahmed M. El-Nahas', 'Joseph O. Hirschfelder', 'C. Pennypacker', 'Aaron L. Halpern', 'Pere Puigserver', 'James G. Hirsch', 'J. S. Binkley', 'William R. McCabe', 'Andrew H. Wyllie', 'Morton Kramer', 'Alexander Parkhomenko', 'Kenneth Macleod', 'Cyrus Chothia', 'A. H.-J. Wang', 'Helmut Blöcker', 'Tong Ihn Lee', 'Hans-Gerd Lipinski', 'Mary Jane Ferraro', 'Michael Zuker', 'D. C. Wiley', 'Marc Zabeau', 'Jeremy Peterson', 'Henderson Ic', 'Insa Gathmann', 'Patrice J. Morin', 'D. Easton', 'F. Barre-Sinoussi', 'Rodney A. Brooks', 'P.-G. De Gennes', 'Jean Morissette', 'David J. Lockhart', 'Betty J. Flehinger', 'Robert H. Waterston', 'Karl A. Piez', 'Morten Kjeldgaard', 'C Rose', 'Michael E. Greenberg', 'Kazuhiko Kawasaki', 'Marius Wernig', 'Margherita Maffei', 'William J. Martone', 'Kiran Garimella', 'A. Naja', 'Jane G. Schaller', 'Ralph S. Paffenbarger', 'Masatoshi Nei', 'Randall K. Saiki', 'Michael Polanyi', 'Carl R. Woese', 'Nicola Mulder', 'Ilya Spigland', 'Stephen F. Jencks', 'Michael C. Wendl', 'Robert L. Spitzer', 'Marcy Silver', 'Marcus E. Raichle', 'George Scatchard', 'Leonid Kruglyak', 'Lynn R. Sykes', 'Ali Zandieh', 'Kristian Cibulskis', 'Peter Philippsen', 'Joseph Sambrook', 'Robert Kamen', 'James I. Cleeman', 'Hiroaki Hemmi', 'Lucinda Fulton', 'Carole A. Cull', 'David W. Hosmer', 'P. Andrew Futreal', 'Edgar F. Meyer', 'T. J. Sejnowski', 'Paul W. Gast', 'John C. Khoo', 'Lee N. Robins', 'Webb Miller', 'Rory Collins', 'Chad Nusbaum', 'Lynn M. Schriml', 'Martin Poenie', 'Joseph Berkson', 'Eric R. Fearon', 'Raymond D. Miller', 'Sidney P. Colowick', 'Jay D. Pearson', 'Marie-Angèle Morel']

['David P. Reed',
 'Mikulas Popovic',
 'Katherine M. Flegal',
 'Boris Reva',
 'Emily S. Winn-Deen',
 'B. G. Barrell',
 'Richard G. Margolese',
 'John M. Chirgwin',
 'Floyd Ratliff',
 'Joan Massagué',
 'Sarah Milne',
 'M. Overgaard',
 'Joseph L. Goldstein',
 'Todd M. Lowe',
 'Jonathan Marchini',
 'Peter J. Ramadge',
 'Daniel R. Marshak',
 'Stephen M. J. Searle',
 'Leon Festinger',
 'James Kennedy',
 'Michael B. Mock',
 'Juha Kononen',
 'Julia B. Lewis',
 'Jonathan R. Edwards',
 'Victor E. Velculescu',
 'Peter J. Turnbaugh',
 'Akira Nagafuchi',
 'Amy E. Pasquinelli',
 'Norman Arnheim',
 'Thomas D. Schmittgen',
 'Linda M. Rolnitzky',
 'Rodrigo Lopez',
 'J.-C. Gazet',
 'Joel S. Hayflick',
 'P. C. Eklund',
 'Stefanie S. Jeffrey',
 'Camille Parmesan',
 'N. Cabrera',
 'Andrew J. Mungall',
 'Robert Baertsch',
 'Robert E. Bruccoleri',
 'Robert S. Pinals',
 'Helen Skaletsky',
 'Keith Nelson',
 'J. Richard Hackman',
 'J. Michael Cherry',
 'Mark F. Pittenger',
 'Richa Agarwala',
 'Mark A. DePristo

In [26]:
# Disruption
query = """
call apoc.export.csv.query(
"MATCH (a:Top42Author)
WHERE a.last_author_yr > 2013 

MATCH (a)-[:AUTHORED]->(q:Quanta)<-[:AUTHORED]-(b:Author)
WHERE q.venue in ['Angewandte Chemie','Blood','Cancer Cell','Cancer Discovery','Cancer Research','Cell','Cell Host & Microbe','Cell Metabolism','Cell Stem Cell','Chemistry & Biology','The EMBO Journal','Genes & Development','Immunity','Journal of Neurology','Journal of the American Chemical Society','JAMA','Journal of Biological Chemistry','Journal of Cell Biology','Journal of Clinical Investigation','Journal of Experimental Medicine','Journal of Medicinal Chemistry','The Lancet','Nature Cell Biology','Nature Chemical Biology','Nature Chemistry','Nature Medicine','Nature Methods','Nature','Nature Biotechnology','The New England Journal of Medicine','Neuron','Nature Genetics','Nature Immunology','Nature Neuroscience','Nature Structural & Molecular Biology','PLOS Biology','PLOS Genetics','PLOS Pathogens','Proceedings of the National Academy of Sciences of the United States of America','Science Signaling','Science Translational Medicine','Science']
    AND q.year < a.last_author_yr
    AND b.name in ['David P. Reed', 'Mikulas Popovic', 'Katherine M. Flegal', 'Boris Reva', 'Emily S. Winn-Deen', 'B. G. Barrell', 'Richard G. Margolese', 'John M. Chirgwin', 'Floyd Ratliff', 'Joan Massagué', 'Sarah Milne', 'M. Overgaard', 'Joseph L. Goldstein', 'Todd M. Lowe', 'Jonathan Marchini', 'Peter J. Ramadge', 'Daniel R. Marshak', 'Stephen M. J. Searle', 'Leon Festinger', 'James Kennedy', 'Michael B. Mock', 'Juha Kononen', 'Julia B. Lewis', 'Jonathan R. Edwards', 'Victor E. Velculescu', 'Peter J. Turnbaugh', 'Akira Nagafuchi', 'Amy E. Pasquinelli', 'Norman Arnheim', 'Thomas D. Schmittgen', 'Linda M. Rolnitzky', 'Rodrigo Lopez', 'J.-C. Gazet', 'Joel S. Hayflick', 'P. C. Eklund', 'Stefanie S. Jeffrey', 'Camille Parmesan', 'N. Cabrera', 'Andrew J. Mungall', 'Robert Baertsch', 'Robert E. Bruccoleri', 'Robert S. Pinals', 'Helen Skaletsky', 'Keith Nelson', 'J. Richard Hackman', 'J. Michael Cherry', 'Mark F. Pittenger', 'Richa Agarwala', 'Mark A. DePristo', 'James H. Ware', 'Serafim Batzoglou', 'Lee A. Green', 'J. E. Till', 'Elizabeth Ward', 'Sheila Scully', 'William T. Friedewald', 'Allan M. Skene', 'V. Narry Kim', 'Tina Graves', 'Elaine R. Mardis', 'George W. Torrance', 'Robert L. Coffman', 'Andreas Heger', 'Ellen E. Walters', 'Jeremy D. Selengut', 'Ruth E. Ley', 'C. V. Shank', 'Gregory J. Hannon', 'Donald S. Rae', 'James A. Stockman', 'Saira Salahuddin', 'J. G. Bednorz', 'D. von Fournier', 'J. F. A. P. Miller', 'Scott Schwartz', 'Ivica Letunic', 'Heike Fiegler', 'J. Gillis', 'Russell Ross', 'Christie Kovar', 'Bruce W. Birren', 'Richard S. Lazarus', 'Helga Boedtker', 'George A. Miller', 'George H. Mead', 'Arthur Purdy Stout', 'LaDeana W. Hillier', 'William A. Sibley', 'Michael P. Bevilacqua', 'Michele L. Pearson', 'Jean-Roger Le Gall', 'M. R. Cooper', 'Jean Thierry-Mieg', 'Joanna E. Siegel', 'Raffaella Sordella', 'David J. Spiegelhalter', 'Frans Bruyninckx', 'Bruce M. Spiegelman', 'George G. J. M. Kuiper', 'Stacey Gabriel', 'Seymour S. Kety', 'Kenneth F. Schulz', 'S. L. McCall', 'Morris W. Foster', 'Huaiyu Mi', 'Nick Patterson', 'Donna K. Slonim', 'Richard H. Chapman', 'Clare Seamark', 'Saul A. Teukolsky', 'Craig C. Mello', 'E Robinson', 'A E Weyman', 'Hartmut Land', 'Ramon Parsons', 'Katsuaki Hoshino', 'George J. Taylor', 'Deborah Wentworth', 'M Richard', 'Thomas E. Carew', 'David J. Lipman', 'Erich Bluhmki', 'John D. McPherson', 'Robert J. Garrison', 'William F. Novotny', 'Kary B. Mullis', 'Robert M. Weisskoff', 'Zemin Ning', 'Daniel Shoemaker', 'M Bond', 'Jillian Boreham', 'Zoltán N. Oltvai', 'Jay R. Ritter', 'Leland H. Hartwell', 'G. Binnig', 'Tomaso Poggio', 'Nathan Maccoby', 'Donald D. Newmeyer', 'Benjamin A. Barnes', 'Charles Tanford', 'M. Michele Manos', 'Otto A. Bessey', 'Yudong D. He', 'Herbert A. Simon', 'Deborah Estrin', 'John V. Moran', 'Gabor Gyapay', 'Michael Shi', 'Edward J. Roccella', 'Richard D. Piner', 'Taro Kawai', 'Erica Sodergren', 'Stephen J. Aldington', 'M. Y. Kim', 'Gilean McVean', 'John P. Perdew', 'Scott Shenker', 'S. Moncada', 'Anders Krogh', 'Jerome Cornfield', 'Ralph M. Steinman', 'M. E. J. Newman', 'Jeffrey C. Lee', 'Benjamin D. Hall', 'Miriam J. Alter', 'Sidney Udenfriend', 'James K. V. Willson', 'Mark S. Guyer', 'Julian Peto', 'James E. Galagan', 'Alice C. Young', 'Theophil Staehelin', 'Steven D. Averbuch', 'Edmund G. Lowrie', 'Cliff Tabin', 'P Sandercock', 'Nathan Goodman', 'Richard A. Shafer', 'Steve Rozen', 'A. R. Hunt', 'J.A. van Dongen', 'Mary C. Betlach', 'Michael C. Zody', 'Harry Towbin', 'Myunghee C. Paik', 'Mark Diekhans', 'Michael Gent', 'Mark S. Boguski', 'Hongjie Dai', 'David V. Goeddel', 'Andrew Neil', 'Yunzhen Cao', 'M. H. Gault', 'Paul Meier', 'Robert E. Kleiger', 'Kim Rutherford', 'William R. Pearson', 'Janan T. Eppig', 'Bert Sakmann', 'Stuart McLaren', 'R Parker', 'James H. Jandl', 'David P. Bartel', 'C Redmond', 'Jerome Vinograd', 'Mark Dickson', 'Barry A. Morgan', 'Julie D. Thompson', 'Mike Rothe', 'Tobias Doerks', 'Jeffrey G. Reid', 'D. J. E. Ingram', 'Yuling Hong', 'Barbara J. McNeil', 'Robert Tibshirani', 'Jeremy H. Burroughes', 'Ueli Gubler', 'C. Milstein', 'Paul J. Flory', 'A. K. Mitchell', 'Mark T. Ross', 'I. M. Stratton', 'P. D. Home', 'Andrew P. Holmes', 'C. Arden Pope', 'Tim R. Mosmann', 'Richard Peto', 'Richard C. Graham', 'John M. Leonard', 'Arthur B. Lamb', 'Benoit B. Mandelbrot', 'David E. Goldberg', 'E. Kellenberger', 'F. Schedin', 'C. K. Osborne', 'Nicholas Kassebaum', 'Ch. Gerber', 'Russell Turner', 'Stéphane Guindon', 'Frank E. Speizer', 'Susan G. Arbuck', 'Thomas D. Willis', 'R W Davis', 'Phillip D. Zamore', 'Nan M. Laird', 'Barth Hoogstraten', 'Kim C. Worley', 'Hugo Muench', 'Douglas Hanahan', 'Margaret D. Carroll', 'Gary Ruvkun', 'B. E. Warren', 'Catherine A. Lozupone', 'Robert F. Meenan', 'Naruya Saitou', 'James S. Marks', 'Wilson Ia', 'Minze Stuiver', 'Seymour Benzer', 'Terrence S. Furey', 'Judah Folkman', 'Richard E. Nisbett', 'Pui Yan Kwok', 'Mark A. Saper', 'Erika Schmid', 'Marita Cross', 'Dennis G. Ballinger', 'Elisabeth Burdick', 'D. Wayne Taylor', 'Michele Cargill', 'Fred Ederer', 'Loren F. Hiratzka', 'Oliver Schwardt', 'Saul Genuth', 'Stuart Bunting', 'Arnold Oliphant', 'N. E. Day', 'Paul Predki', 'Paul D. Sorlie', 'Donna M. Rennick', 'Louis A. Tartaglia', 'Parnesh Raniga', 'Robert Axelrod', 'A. McGuire', 'Stewart Scherer', 'Lucian L. Leape', 'U. J. Lewis', 'J Burns', 'Sarah E. Fowler', 'Bernard B. Brodie', 'Ghia Euskirchen', 'Kate Downes', 'Scott M. Hammond', 'Robert D. Fleischmann', 'Robert M. Kuhn', 'Richard J. Brand', 'Jerome Y. Lettvin', 'Thomas R. Fleming', 'Joseph D. Mosca', 'Erin Pleasance', 'Nahida H. Gordon', 'Calvin B. Harley', 'Russell Higuchi', 'Réka Albert', 'Keaven M. Anderson', 'J. F. Forbes', 'M Fernandez', 'Zhiping Gu', 'Bala Swaminathan', 'Jeffrey Vieira', 'Annie Yang', 'P. B. Hirsch', 'R. S. J. Frackowiak', 'Tavia Gordon', 'Ralf W. Grosse-Kunstleve', 'James R. Rice', 'V. S. Kalyanaraman', 'Daniel C. Koboldt', 'James B. Hicks', 'Jerome S. Bruner', 'Elia Stupka', 'Michael C. Jensen', 'Randi Y. Leavitt', 'Stephen F. Altschul', 'Hidemi Watanabe', 'Andrey Sivachenko', 'William J. Rutter', 'Michael Fiorino', 'Louis Fehrenbacher', 'James J. Gibson', 'Béatrice Vanbervliet', 'Richard C. Eastman', 'Qui-Lim Choo', 'Rodger Staden', 'Jeffrey C. Barrett', 'Donald B. Rubin', 'Fredrik Ronquist', 'Fuli Yu', 'Reubin Andres', 'Jacob Cohen', 'Joseph H. Roe', 'Troy Moore', 'David Tilman', 'Zhongwu Lai', 'Jonathan Sebat', 'Wonhee Jang', 'David W. Bilheimer', 'P. Armitage', 'Kalevi Pyörälä', 'John E. Ware', 'J. J. Hopfield', 'Norman E. Breslow', 'Julius Marmur', 'J. D. A. Van Embden', 'Christoph Kalka', 'Eric S. Lander', 'George L. Gabor Miklos', 'Douglas B. Rusch', 'Katherine S. Elliott', 'Alan Coulson', 'Jeffrey N. Rottman', 'Scott V. Tingey', 'Arne Tiselius', 'Dennis A. Gilbert', 'Wendy Winckler', 'Sean Humphray', 'Max Halperin', 'E. Wolf', 'Douglas L. Johnson', 'R. C. Turner', 'Arthur M. Sobol', 'Cladd E. Stevens', 'S. Cobbe', 'E. P. Greenberg', 'Howard H. Hiatt', 'P. J. Randle', 'E. Orowan', 'Aaron T. Beck', 'M Klagsbrun', 'Yves-Alain Barde', 'Daniel Kahneman', 'Robert T. DeBoy', 'G. E. Palade', 'Adam Frankish', 'Sam T. Roweis', 'Andersen Kw', 'Thomas J. Dull', 'Michaele C. Christian', 'David R. Cox', 'Donald Geman', 'M. H. F. Wilkins', 'William J. Henzel', 'Brygg Ullmer', 'Carol Redmond', 'Michael B. Eisen', 'Chang Gong Liu', 'Paul Milgrom', 'Trampisch Hj', 'M. D. Poulik', 'Daniel W. Bradley', 'Carl-Magnus Rudenstam', 'Albert László Barabási', 'David S. Hogness', 'Michael L. Metzker', 'Donald Pearson', 'J. S. Turner', 'James F. Fries', 'Suzannah Bumpstead', 'Ben Z. Locke', 'Marianne Kearney', 'R. Mead', 'Tim Hubbard', 'William Saurin', 'R. R. Ernst', 'Federico Girosi', 'Peter J. Sabo', 'Alex Bateman', 'Dienstag Jl', 'Ed Harlow', 'Carrie Sougnez', 'Guy Plunkett', 'Richard Bellman', 'Aaron McKenna', 'Gary J. Olsen', 'Lawrence L. Kupper', 'Mark D. Adams', 'Luther T. Clark', 'Janet M. Corrigan', 'A. F. Huxley', 'Nigel P. Carter', 'D Wright', 'William H. Stein', 'Robert S. Fulton', 'Heidi Greulich', 'Sankar Swaminathan', 'Toby J. Gibson', 'Simon G. Gregory', 'Aravind Subramanian', 'C. Katz', 'Tetsuo Maita', 'Helena Hämäläinen', 'Eugene W. Myers', 'Rhian Gwilliam', 'Anup Madan', 'Forrest E. Kendall', 'Cynthia L. Ogden', 'Aileen Robertson', 'Harold T. Dodge', 'Marilyn Bergner', 'D. R. Cox', 'G. I. Taylor', 'Jonathan Butler', 'Andrew B. Lassar', 'S Mason', 'Roland M. H. Schein', 'Margo A. Denke', 'Richard M. Goldstein', 'Robert B. Catalano', 'Alexander E. Urban', 'Harvey J. Alter', 'Monique G. Lê', 'Gyorgy Hutvagner', 'Darren A. Natale', 'Stanley J. Korsmeyer', 'J. Craig Venter', 'D Nelson', 'Richard Tizard', 'W Jack', 'Lee Rowen', 'Hailing Hsu', 'Robert P. Gaynes', 'Donald R. Huffman', 'Richard H. Creech', 'Michael H. Kutner', 'Michael A. Saunders', 'R Peto', 'Carol J. Bult', 'Ian Korf', 'D. R. Curtis', 'Michael Held', 'Michael R. Hee', 'Ali A. Mokdad', 'Frank Delaglio', 'Donald S. Fredrickson', 'Jeffrey S. Beck', 'Shirley Zhu', 'Jorge J. Yunis', 'Richard A. Olshen', 'David Altshuler', 'Gabriele Bergers', 'Sandra W. Clifton', 'Colin N. Dewey', 'Georges Köhler', 'Paul T. Spellman', 'Stanley Lemeshow', 'Dominic A. Scudiero', 'Basil M. Rifkind', 'Walter M. Fitch', 'Endel Tulving', 'J. M. Bland', 'Nikola P. Pavletich', 'Nawab Qizilbash', 'Raymond J. MacDonald', 'W. H. Zachariasen', 'Daniel T. Colbert', 'D. R. Matthews', 'Patrick Brill-Edwards', 'Michael K. Showe', 'Marc van de Wetering', 'Bela Julesz', 'Earl Hubbell', 'Scott L. Zeger', 'Peter K. Wellauer', 'Zbyszek Otwinowski', 'Charles T. Kresge', 'Helen Van Vunakis', 'Nicoletta Sacchi', 'Douglas G. Altman', 'R Doll', 'Richard W. Reynolds', 'A. Klug', 'Amos Tversky', 'Quinn L. Deveraux', 'Patrick O. Brown', 'Patrick D. Wall', 'David Botstein', 'K Matsushima', 'Brufman G', 'Zhaodan Cao', 'Elisabeth Buchdunger', 'Alex Pentland', 'F. P. Bowden', 'J Holmes', 'Kenneth J. Livak', 'K Sakai', 'Hoda Khouri', 'Jerzy Neyman', 'R. Willox', 'Raymond G. Goodwin', 'Richard Durbin', 'Alan J. Buckler', 'Fu Lu', 'Gordon J', 'Pablo Tamayo', 'Shimon Ullman', 'Gary W. Trucks', 'Morris J. Karnovsky', 'J. S. Waugh', 'Wen Lin Kuo', 'Nicholas B. Lydon', 'Amy J. Weiner', 'Anne S. Olsen', 'Allan C. Wilson', 'Daniel H. Haft', 'Stephen E. Haynesworth', 'Marek Kubin', 'Carol Churcher', 'Todd D. Taylor', 'Jane Rogers', 'Teresa C. Horan', 'S. Korzeniowski', 'Nick Goldman', 'John P. Huelsenbeck', 'Huyen Dinh', 'Maynard V. Olson', 'Vern Paxson', 'Joel E. Richardson', 'John C. Cassel', 'Josep F. Abril', 'Robert F. Kibler', 'H. Gervásio', 'David I. Gottlieb', 'Sarah Edkins', 'Howard K. Schachman', 'David Beach', 'Andrew P. Feinberg', 'Hilary King', 'John C. Doyle', 'Jon Teague', 'Laurence Kennedy', 'Anthony R. Kerlavage', 'Drummond Rennie', 'Allyson Ross Davies', 'George Kuo', 'Jian Bing Fan', 'Gwyneth Van', 'Paul McGale', 'Ellson Y. Chen', 'Trevor Hastie', 'Pierre R. Villeneuve', 'Thomas Tuschl', 'Richard R. Copley', 'Torben Palshof', 'James R. Hudson', 'Sandra Silberman', 'John McCarthy', 'Wieslaw J. Roth', 'Douglas J. Schwartzentruber', 'Ian Dunham', 'Carol W. Greider', 'Leslie L. Iversen', 'Sean R. Eddy', 'Claudia A. Seipp', 'Frederic M. Richards', 'Lawrence G. Hunsicker', 'Rajinder Kaul', 'Mark R. Chassin', 'Susan E. Folstein', 'Richard Breathnach', 'Thomas R. Dawber', 'E. A. McCulloch', 'Stephen C. Reingold', 'Mukesh Dherani', 'F Taylor', 'Santos A. Susin', 'Simon Myers', 'Ronald H. Reeder', 'Hilary A. Coller', 'James E. Grizzle', 'Lynn A. G. Ries', 'Eugene L. Brown', 'Robert Karasek', 'Edward Witten', 'Myron Flickner', 'Oliver H. Lowry', 'A. L. Hodgkin', 'Kenneth J. Arrow', 'Jorja G. Henikoff', 'G. E. Hutchinson', 'Nancy A. Thornberry', 'Naoufal Zamzami', 'A. P. Alivisatos', 'Vincent P. Stanton', 'E Thomson', 'Deanna M. Church', 'Mark S. Chee', 'Joe M. McCord', 'François Artiguenave', 'Liisa Holm', 'Eladio Viñuela', 'Mary Burden', 'Granger G. Sutton', 'Oliver Smithies', 'Herbert W. Boyer', 'William H. Habig', 'J Graham', 'J. R. Vane', 'Therese Sørlie', 'Audrey Duncanson', 'Marjorie B. Lees', 'Franklin Kim', 'Maurizio Belfiglio', 'C Davies', 'Edmund A. Gehan', 'Jill P. Mesirov', 'Alastair Kent', 'John R. Glover', 'Susan M. Hailpern', 'Patrick Therasse', 'Sarah Hunt', 'Arthur L. Delcher', 'Gavin Sherlock', 'Martin Gibson', 'Robert P. Abelson', 'Kristin Ardlie', 'Timothy B. Stockwell', 'Zhengdong D. Zhang', 'Stewart Craig', 'Charles Dauguet', 'Thomas Jenuwein', 'Dean Burk', 'Joseph Ragaz', 'Renaud Capdeville', 'Halsted R. Holman', 'Mark H. Skolnick', 'Harold Swerdlow', 'Jennifer Baldwin', 'Roger E. Kirk', 'Erik L. L. Sonnhammer', 'Adam Auton', 'Abraham Kagan', 'Marianne Sullivan', 'Torsten N. Wiesel', 'Vishwanath R. Iyer', 'W. Brian Arthur', 'Michael L. Watson', 'Darrel A. Regier', 'Graeme A. Reid', 'D. H. Auston', 'Srinivasa M. Srinivasula', 'Daniel I. Arnon', 'J Clarke', 'Marshal F. Folstein', 'Tullia Lindsten', 'Robert J. Lipshutz', 'Lucy Matthews', 'Edwin M. Southern', 'William B. Kannel', 'James C. Mullikin', 'Matthew Stephens', 'Jane Grimwood', 'Ross L. Prentice', 'Laura Elnitski', 'Alexandre Montpetit', 'M. G. Bawendi', 'Nathan O. Kaplan', 'Jim Gray', 'Elizabeth Lacy', 'Mark Yandell', 'Christopher B. Burge', 'Andrew Kasarskis', 'Alan T. Bankier', 'Clifford L. Johnson', 'Koichiro Tamura', 'Harold Weintraub', 'L. Di Martino', 'Robert A. Weinberg', 'Susan L. Naylor', 'Andrew Wakeham', 'Karin Eiglmeier', 'John G. Kirkwood', 'Scott Uknes', 'Steven E. Scherer', 'Jacques Ferlay', 'David L. Donoho', 'Timothy Fennell', 'Genji Matsuda', 'Barbara E. Stranger', 'Eric A. Jaffe', 'Daphne W. Bell', 'D. Williams Parsons', 'John V. Bennett', 'Matt van de Rijn', 'Parkhurst A. Shore', 'Paul Liska', 'Aram V. Chobanian', 'James J. Heckman', 'Simon L. Howell', 'R. W. Butcher', 'S. M. Zahurak', 'Pamela Whittaker', 'Robert A. Rosati', 'Sarah Nutland', 'David Torrents', 'Yoav Freund', 'Gabor T. Marth', 'Catherine A. Ball', 'T. N. Bhat', 'Mark J. Daly', 'Eric A. Swanson', 'Scott L. Weinrich', 'Robert Katzman', 'Taylor A. Sohn', 'Erding Hu', 'Andreas Thess', 'Sj Wilson', 'Ants Leetmaa', 'P. F. Scholander', 'Allen Newell', 'Argiris Efstratiadis', 'John P. Feighner', 'Lisa French', 'Larry Rubinstein', 'A. L. Page', 'Catherine Michaud', 'Jean Endicott', 'Jacques Monod', 'Anne Dornhorst', 'Kara Dolinski', 'Stanley L. Wallace', 'Kurt J. Greenlund', 'Emily Bernstein', 'Francine Brière', 'Ronald Melzack', 'Gerald M. Rubin', 'Richard Helm', 'C Fox', 'Robert G. Parr', 'David B. Johnson', 'Lon R. Cardon', 'Gail R. Martin', 'David L. Swofford', 'Roland Heilig', 'Helen Goodell', 'John H. Holland', 'Francis M. Miezin', 'Frederick Mosteller', 'J.Ward Kennedy', 'Sidney J. Cutler', 'Michael S. Gottlieb', 'John R. Marler', 'Keith J. Worsley', 'Waldo E. Cohn', 'Joan Keesey', 'Andrew Y. Ng', 'Brian Haynes', 'U. Veronesi', 'Warren J. Hehre', 'Allan Van Oosterom', 'Martin Fishbein', 'Arian Smit', 'James B. Wyngaarden', 'Robin Humphry-Baker', 'David G. Lowe', 'Charles D. Keeling', 'Alan D. Lopez', 'Kimmen Sjölander', 'David G. Kleinbaum', 'W. James Kent', 'J Martin Bland', 'Robert J. Dodson', 'K. S. Novoselov', 'Mercedes Camalli', 'Alexej Abyzov', 'Melvin P. Judkins', 'William H. Majoros', 'D. Lawrence Wickerham', 'Paul Doty', 'Ewan Birney', 'Daniel W. Jones', 'Kung-Yee Liang', 'Gordon Saksena', 'Solange Chamaret', 'Frederick C. Neidhardt', 'John Nash', 'Michael S. Waterman', 'Thomas N. Sato', 'M. Castiglione', 'Geoffrey E. Hinton', 'Wolfgang Kabsch', 'David Julius', 'David L. Pearle', 'Dennis Y. Loh', 'Ronald A. Laskey', 'Thomas C. Chalmers', 'Noriko Sakaguchi', 'Eric Pelletier', 'Jeffrey A. Cutler', 'Thomas J. Palker', 'Michael D. McLellan', 'M. G. Sarngadharan', 'Michael Feolo', 'Taylor Ja', 'Valerie A. Fadok', 'Andreas Rump', 'F. C. Greenwood', 'Vladimir Korinek', 'Reto Ruedy', 'Lore Zech', 'A. C. T. North', 'Peter W. J. Rigby', 'Donald P. Nierlich', 'Edgardo Lander', 'Jeremy D. Glasner', 'Sarah S. Murray', 'Rebecca DerSimonian', 'Clifton F. Mountain', 'David R. Bentley', 'R. A. Fisher', 'Andrew Fire', 'Gordon L. Shulman', 'Anita L. Stewart', 'Ahmedin Jemal', 'Desmond G. Higgins', 'John M. Darley', 'H. J. Stewart', 'Isaac L. Chuang', 'David J. Dooling', 'David W. Tank', 'Yuval Eshed', 'Monica Williams', 'Kikuko Hotta', 'Nj Bell', 'Kazuto Kato', 'Linus Pauling', 'Freddie Bray', 'Bert Vogelstein', 'K McPherson', 'Werner Risau', 'Evan Mauceli', 'Federica Sallusto', 'Charles Atkinson', 'Paul Emsley', 'D. C. Van Essen', 'P Park', 'John D. Lafferty', 'Donna F. Stroup', 'Mark Weiser', 'Charles H. Hennekens', 'Beverley Shields', 'Hugh Paterson', 'Ken Dewar', 'John E. Helzer', 'Sanford L. Palay', 'George Paxinos', 'Liesi E. Hebert', 'Robert C. Onofrio', 'Rachel M. Freathy', 'David A. Hungerford', 'Yasunori Ogura', 'Bruce N. Ames', 'Sam Griffiths-Jones', 'Catrina Fronick', 'Ivan Lieberburg', 'Stephen T. Sherry', 'Charmaine D. Royal', 'N. F. Mott', 'Douglas T. Ross', 'H. Murayama', 'Marshall N. Rosenbluth', 'Rodney M. Hewick', 'W. M. Cowan', 'Karen Hsiao', 'Timo T. Valle', 'Yasushi Totoki', 'Amos Marc Bairoch', 'William A. Knaus', 'William J. Fulkerson', 'Christopher K. Raymond', 'Ross A. Lippert', 'John A. Pople', 'W. Richard McCombie', 'Lora Lewis', 'Michael W. Hunkapiller', 'B. Gutenberg', 'Karen Eilbeck', 'John R. Carson', 'Jeremy White', 'Shelley E. Taylor', 'Tarjei S. Mikkelsen', 'Vincent Ferretti', 'Thomas H. Holmes', 'Karsten Hokamp', 'David Haussler', 'Graham R. Bignell', 'W. Dansgaard', 'Paul Ekman', 'Amy A. Caudy', 'Ingrid Daubechies', 'B. A. Roe', 'A. Howie', 'Joachim Messing', 'Franz Josef Och', 'Eli Robins', 'Bates Dw', 'H. R. Horvitz', 'Michael Ashburner', 'Hamilton O. Smith', 'William N. Anderson', 'Kenneth W. Kinzler', 'Joseph H. Connell', 'Tom Maniatis', 'John M.C. Gutteridge', 'Suzanna E. Lewis', 'Rolf Apweiler', 'Robert I. Levy', 'Paul J. Deutsch', 'Eric W. Wong', 'Robin Milner', 'Keith R. Porter', 'Stefan Taudien', 'S Takashima', 'Michael J. Thun', 'Joe Sambrook', 'E. A. Clarke', 'David W. Leung', 'Marco Punta', 'M. Karplus', 'B. F. Skinner', 'Vishva M. Dixit', 'Andrew G. Fraser', 'Brian R. Lawn', 'Max H. Myers', 'Sewall Wright', 'Martine Van Glabbeke', 'Bruce Culleton', 'Martin L. Hooper', 'Nancy F. Hansen', 'Albert J. Belanger', 'Kirk R. Thomas', 'Andrew Menzies', 'Stanford Moore', 'Miller Ja', 'Max Born', 'Ulrich K. Laemmli', 'Walter Gilbert', 'Tracy M. Bryan', 'G. Flandrin', 'Alfonse T. Masi', 'Giorgio Cocconi', 'James H. Stock', 'Nathan Mantel', 'Kenneth P. Burnham', 'Lawrence E. Shulman', 'Sheila Seal', 'David S. Johnson', 'Gonçalo R. Abecasis', 'Donald E. Groom', 'Ramzi S. Cotran', 'S Clarke', 'Owen P. Hamill', 'A. K. Geim', 'William P. Castelli', 'Jinghui Zhang', 'Folke Skoog', 'Alejandro A. Schäffer', 'George E. Palade', 'Paul E. Meehl', 'Lee Murphy', 'Antman Em', 'René O. Mirimanoff', 'Bonnee Rubinfeld', 'Jennifer J. Anderson', 'Gitanjali M. Singh', 'Anthony J. Cox', 'Richard O. Russell', 'David L. Sackett', 'Felsenfeld Al', 'Jane Peterson', 'H. Craig', 'M Schumacher', 'Sergey V. Morozov', 'Jacqueline Kosecoff', 'P. J. Flory', 'Irving Langmuir', 'Albert H. Coons', 'Roel P. Funke', 'Steven L. Salzberg', 'Alane Gray', 'James W. Curran', 'Lars Hedin', 'G. A. R. Johnston', 'G. Bonadonna', 'Auke Tellegen', 'Michael R. Brent', 'al. et', 'Albert-László Barabási', 'Earl W. Sutherland', 'William Haenszel', 'Dmitriy A. Dikin', 'J. P. Grime', 'N. Ferrara', 'A. D. Bangham', 'Karen A. Ketchum', 'Sidney C. Smith', 'Douglas P. Wagner', 'Don W. Fawcett', 'Steven H. Strogatz', 'Jonathan K. Pritchard', 'Toby Bloom', 'Deborah R. Shapiro', 'Phyllis A. Wingo', 'Ronald L. Rivest', 'I. V. Grigorieva', 'Mary Osborn', 'Glenn Tesler', 'P. J. Sellers', 'F. Sanger', 'David H. Gelfand', 'Howard A. Eder', 'J. L. Weiland', 'Jennifer R. Wortman', 'Daniel McGee', 'G. D. Farquhar', 'John F. Heidelberg', 'Yuen Shing', 'Brenda K. Edwards', 'John Bowlby', 'Ruslan Medzhitov', 'Ching W. Tang', 'Norman A. Doggett', 'Gerald R. Fink', 'Trevor Hawkins', 'Samuel B. Guze', 'Richard R. Hudson', 'Solomon A. Berson', 'Eugene R. Passamani', 'David B. Jaffe', 'Roger Y. Tsien', 'Bigger Jt', 'Victor R. Ambros', 'David H. Culver', 'Donald D. Van Slyke', 'John E. Wennberg', 'McGuire Wl', 'Wayne J. Higgins', 'Jingtao Sun', 'Reginald A. Clift', 'Jitendra Malik', 'Gordon G. Wong', 'Fred Ramsdell', 'Heather Schmidt', 'J. S. Bell', 'P. G. Smith', 'Michael L. Frigge', 'Gilbert Chu', 'Irene M. Stratton', 'Martin Ringwald', 'Katherine W. Timothy', 'William H. Dietz', 'Joseph L. Fleiss', 'E Howard', 'Yuri Lazebnik', 'Cyrus Levinthal', 'Stephen Frank Schaffner', 'M. A. Fiatarone', 'Cole Trapnell', 'J. Milburn Jessup', 'Chris C A Spencer', 'Richard M. Karp', 'Henry A. Erlich', 'Jörg Schultz', 'Gary Struhl', 'Steven R. Gill', 'Patrick Minx', 'Craig H. Mermel', 'L. J. Ignarro', 'Kevin McKernan', 'Ahmed M. El-Nahas', 'Joseph O. Hirschfelder', 'C. Pennypacker', 'Aaron L. Halpern', 'Pere Puigserver', 'James G. Hirsch', 'J. S. Binkley', 'William R. McCabe', 'Andrew H. Wyllie', 'Morton Kramer', 'Alexander Parkhomenko', 'Kenneth Macleod', 'Cyrus Chothia', 'A. H.-J. Wang', 'Helmut Blöcker', 'Tong Ihn Lee', 'Hans-Gerd Lipinski', 'Mary Jane Ferraro', 'Michael Zuker', 'D. C. Wiley', 'Marc Zabeau', 'Jeremy Peterson', 'Henderson Ic', 'Insa Gathmann', 'Patrice J. Morin', 'D. Easton', 'F. Barre-Sinoussi', 'Rodney A. Brooks', 'P.-G. De Gennes', 'Jean Morissette', 'David J. Lockhart', 'Betty J. Flehinger', 'Robert H. Waterston', 'Karl A. Piez', 'Morten Kjeldgaard', 'C Rose', 'Michael E. Greenberg', 'Kazuhiko Kawasaki', 'Marius Wernig', 'Margherita Maffei', 'William J. Martone', 'Kiran Garimella', 'A. Naja', 'Jane G. Schaller', 'Ralph S. Paffenbarger', 'Masatoshi Nei', 'Randall K. Saiki', 'Michael Polanyi', 'Carl R. Woese', 'Nicola Mulder', 'Ilya Spigland', 'Stephen F. Jencks', 'Michael C. Wendl', 'Robert L. Spitzer', 'Marcy Silver', 'Marcus E. Raichle', 'George Scatchard', 'Leonid Kruglyak', 'Lynn R. Sykes', 'Ali Zandieh', 'Kristian Cibulskis', 'Peter Philippsen', 'Joseph Sambrook', 'Robert Kamen', 'James I. Cleeman', 'Hiroaki Hemmi', 'Lucinda Fulton', 'Carole A. Cull', 'David W. Hosmer', 'P. Andrew Futreal', 'Edgar F. Meyer', 'T. J. Sejnowski', 'Paul W. Gast', 'John C. Khoo', 'Lee N. Robins', 'Webb Miller', 'Rory Collins', 'Chad Nusbaum', 'Lynn M. Schriml', 'Martin Poenie', 'Joseph Berkson', 'Eric R. Fearon', 'Raymond D. Miller', 'Sidney P. Colowick', 'Jay D. Pearson', 'Marie-Angèle Morel']

WITH a, COUNT(q) AS num_top_collabs
WHERE num_top_collabs > 0

MATCH (a)-[:AUTHORED]->(q:Quanta)
WHERE q.venue in ['Angewandte Chemie','Blood','Cancer Cell','Cancer Discovery','Cancer Research','Cell','Cell Host & Microbe','Cell Metabolism','Cell Stem Cell','Chemistry & Biology','The EMBO Journal','Genes & Development','Immunity','Journal of Neurology','Journal of the American Chemical Society','JAMA','Journal of Biological Chemistry','Journal of Cell Biology','Journal of Clinical Investigation','Journal of Experimental Medicine','Journal of Medicinal Chemistry','The Lancet','Nature Cell Biology','Nature Chemical Biology','Nature Chemistry','Nature Medicine','Nature Methods','Nature','Nature Biotechnology','The New England Journal of Medicine','Neuron','Nature Genetics','Nature Immunology','Nature Neuroscience','Nature Structural & Molecular Biology','PLOS Biology','PLOS Genetics','PLOS Pathogens','Proceedings of the National Academy of Sciences of the United States of America','Science Signaling','Science Translational Medicine','Science']
     AND exists(q.Disruption) AND a.last_author_yr <= q.year 
WITH a, COUNT(q) AS num_disrupt, SUM(q.Disruption) AS tot_disrupt, MIN(q.Disruption) AS min_disrupt, MAX(q.Disruption) AS max_disrupt, percentileCont(q.Disruption, 0.5) AS median_disrupt 
RETURN a.last_author_yr AS last_author_yr, 
    a.name AS name, 
    num_disrupt, 
    toFloat(tot_disrupt)/num_disrupt AS avg_disrupt, 
    min_disrupt, 
    max_disrupt, 
    median_disrupt"
, "/data/csv/Disrupt_Early_Profs_3.csv", {batchSize:100, iterateList:true, parallel:true})
"""
df_disruption = query_to_df(query, graph)
# df_disruption.to_csv('~/Workspace/UROP/scaling-science/notebooks/Collaboration/Data/Disrupt_Early_Profs.csv', index = False, encoding = "UTF-8")

Starting query... Done (0.31 minutes).


In [27]:
# Collaboration
avg_degree = 11.327052
query = """
call apoc.export.csv.query(
"MATCH (a:Top42Author)
WHERE a.last_author_yr > 2013 

MATCH (a)-[:AUTHORED]->(q:Quanta)<-[:AUTHORED]-(b:Author)
WHERE q.venue in ['Angewandte Chemie','Blood','Cancer Cell','Cancer Discovery','Cancer Research','Cell','Cell Host & Microbe','Cell Metabolism','Cell Stem Cell','Chemistry & Biology','The EMBO Journal','Genes & Development','Immunity','Journal of Neurology','Journal of the American Chemical Society','JAMA','Journal of Biological Chemistry','Journal of Cell Biology','Journal of Clinical Investigation','Journal of Experimental Medicine','Journal of Medicinal Chemistry','The Lancet','Nature Cell Biology','Nature Chemical Biology','Nature Chemistry','Nature Medicine','Nature Methods','Nature','Nature Biotechnology','The New England Journal of Medicine','Neuron','Nature Genetics','Nature Immunology','Nature Neuroscience','Nature Structural & Molecular Biology','PLOS Biology','PLOS Genetics','PLOS Pathogens','Proceedings of the National Academy of Sciences of the United States of America','Science Signaling','Science Translational Medicine','Science']
    AND q.year < a.last_author_yr
    AND b.name in ['David P. Reed', 'Mikulas Popovic', 'Katherine M. Flegal', 'Boris Reva', 'Emily S. Winn-Deen', 'B. G. Barrell', 'Richard G. Margolese', 'John M. Chirgwin', 'Floyd Ratliff', 'Joan Massagué', 'Sarah Milne', 'M. Overgaard', 'Joseph L. Goldstein', 'Todd M. Lowe', 'Jonathan Marchini', 'Peter J. Ramadge', 'Daniel R. Marshak', 'Stephen M. J. Searle', 'Leon Festinger', 'James Kennedy', 'Michael B. Mock', 'Juha Kononen', 'Julia B. Lewis', 'Jonathan R. Edwards', 'Victor E. Velculescu', 'Peter J. Turnbaugh', 'Akira Nagafuchi', 'Amy E. Pasquinelli', 'Norman Arnheim', 'Thomas D. Schmittgen', 'Linda M. Rolnitzky', 'Rodrigo Lopez', 'J.-C. Gazet', 'Joel S. Hayflick', 'P. C. Eklund', 'Stefanie S. Jeffrey', 'Camille Parmesan', 'N. Cabrera', 'Andrew J. Mungall', 'Robert Baertsch', 'Robert E. Bruccoleri', 'Robert S. Pinals', 'Helen Skaletsky', 'Keith Nelson', 'J. Richard Hackman', 'J. Michael Cherry', 'Mark F. Pittenger', 'Richa Agarwala', 'Mark A. DePristo', 'James H. Ware', 'Serafim Batzoglou', 'Lee A. Green', 'J. E. Till', 'Elizabeth Ward', 'Sheila Scully', 'William T. Friedewald', 'Allan M. Skene', 'V. Narry Kim', 'Tina Graves', 'Elaine R. Mardis', 'George W. Torrance', 'Robert L. Coffman', 'Andreas Heger', 'Ellen E. Walters', 'Jeremy D. Selengut', 'Ruth E. Ley', 'C. V. Shank', 'Gregory J. Hannon', 'Donald S. Rae', 'James A. Stockman', 'Saira Salahuddin', 'J. G. Bednorz', 'D. von Fournier', 'J. F. A. P. Miller', 'Scott Schwartz', 'Ivica Letunic', 'Heike Fiegler', 'J. Gillis', 'Russell Ross', 'Christie Kovar', 'Bruce W. Birren', 'Richard S. Lazarus', 'Helga Boedtker', 'George A. Miller', 'George H. Mead', 'Arthur Purdy Stout', 'LaDeana W. Hillier', 'William A. Sibley', 'Michael P. Bevilacqua', 'Michele L. Pearson', 'Jean-Roger Le Gall', 'M. R. Cooper', 'Jean Thierry-Mieg', 'Joanna E. Siegel', 'Raffaella Sordella', 'David J. Spiegelhalter', 'Frans Bruyninckx', 'Bruce M. Spiegelman', 'George G. J. M. Kuiper', 'Stacey Gabriel', 'Seymour S. Kety', 'Kenneth F. Schulz', 'S. L. McCall', 'Morris W. Foster', 'Huaiyu Mi', 'Nick Patterson', 'Donna K. Slonim', 'Richard H. Chapman', 'Clare Seamark', 'Saul A. Teukolsky', 'Craig C. Mello', 'E Robinson', 'A E Weyman', 'Hartmut Land', 'Ramon Parsons', 'Katsuaki Hoshino', 'George J. Taylor', 'Deborah Wentworth', 'M Richard', 'Thomas E. Carew', 'David J. Lipman', 'Erich Bluhmki', 'John D. McPherson', 'Robert J. Garrison', 'William F. Novotny', 'Kary B. Mullis', 'Robert M. Weisskoff', 'Zemin Ning', 'Daniel Shoemaker', 'M Bond', 'Jillian Boreham', 'Zoltán N. Oltvai', 'Jay R. Ritter', 'Leland H. Hartwell', 'G. Binnig', 'Tomaso Poggio', 'Nathan Maccoby', 'Donald D. Newmeyer', 'Benjamin A. Barnes', 'Charles Tanford', 'M. Michele Manos', 'Otto A. Bessey', 'Yudong D. He', 'Herbert A. Simon', 'Deborah Estrin', 'John V. Moran', 'Gabor Gyapay', 'Michael Shi', 'Edward J. Roccella', 'Richard D. Piner', 'Taro Kawai', 'Erica Sodergren', 'Stephen J. Aldington', 'M. Y. Kim', 'Gilean McVean', 'John P. Perdew', 'Scott Shenker', 'S. Moncada', 'Anders Krogh', 'Jerome Cornfield', 'Ralph M. Steinman', 'M. E. J. Newman', 'Jeffrey C. Lee', 'Benjamin D. Hall', 'Miriam J. Alter', 'Sidney Udenfriend', 'James K. V. Willson', 'Mark S. Guyer', 'Julian Peto', 'James E. Galagan', 'Alice C. Young', 'Theophil Staehelin', 'Steven D. Averbuch', 'Edmund G. Lowrie', 'Cliff Tabin', 'P Sandercock', 'Nathan Goodman', 'Richard A. Shafer', 'Steve Rozen', 'A. R. Hunt', 'J.A. van Dongen', 'Mary C. Betlach', 'Michael C. Zody', 'Harry Towbin', 'Myunghee C. Paik', 'Mark Diekhans', 'Michael Gent', 'Mark S. Boguski', 'Hongjie Dai', 'David V. Goeddel', 'Andrew Neil', 'Yunzhen Cao', 'M. H. Gault', 'Paul Meier', 'Robert E. Kleiger', 'Kim Rutherford', 'William R. Pearson', 'Janan T. Eppig', 'Bert Sakmann', 'Stuart McLaren', 'R Parker', 'James H. Jandl', 'David P. Bartel', 'C Redmond', 'Jerome Vinograd', 'Mark Dickson', 'Barry A. Morgan', 'Julie D. Thompson', 'Mike Rothe', 'Tobias Doerks', 'Jeffrey G. Reid', 'D. J. E. Ingram', 'Yuling Hong', 'Barbara J. McNeil', 'Robert Tibshirani', 'Jeremy H. Burroughes', 'Ueli Gubler', 'C. Milstein', 'Paul J. Flory', 'A. K. Mitchell', 'Mark T. Ross', 'I. M. Stratton', 'P. D. Home', 'Andrew P. Holmes', 'C. Arden Pope', 'Tim R. Mosmann', 'Richard Peto', 'Richard C. Graham', 'John M. Leonard', 'Arthur B. Lamb', 'Benoit B. Mandelbrot', 'David E. Goldberg', 'E. Kellenberger', 'F. Schedin', 'C. K. Osborne', 'Nicholas Kassebaum', 'Ch. Gerber', 'Russell Turner', 'Stéphane Guindon', 'Frank E. Speizer', 'Susan G. Arbuck', 'Thomas D. Willis', 'R W Davis', 'Phillip D. Zamore', 'Nan M. Laird', 'Barth Hoogstraten', 'Kim C. Worley', 'Hugo Muench', 'Douglas Hanahan', 'Margaret D. Carroll', 'Gary Ruvkun', 'B. E. Warren', 'Catherine A. Lozupone', 'Robert F. Meenan', 'Naruya Saitou', 'James S. Marks', 'Wilson Ia', 'Minze Stuiver', 'Seymour Benzer', 'Terrence S. Furey', 'Judah Folkman', 'Richard E. Nisbett', 'Pui Yan Kwok', 'Mark A. Saper', 'Erika Schmid', 'Marita Cross', 'Dennis G. Ballinger', 'Elisabeth Burdick', 'D. Wayne Taylor', 'Michele Cargill', 'Fred Ederer', 'Loren F. Hiratzka', 'Oliver Schwardt', 'Saul Genuth', 'Stuart Bunting', 'Arnold Oliphant', 'N. E. Day', 'Paul Predki', 'Paul D. Sorlie', 'Donna M. Rennick', 'Louis A. Tartaglia', 'Parnesh Raniga', 'Robert Axelrod', 'A. McGuire', 'Stewart Scherer', 'Lucian L. Leape', 'U. J. Lewis', 'J Burns', 'Sarah E. Fowler', 'Bernard B. Brodie', 'Ghia Euskirchen', 'Kate Downes', 'Scott M. Hammond', 'Robert D. Fleischmann', 'Robert M. Kuhn', 'Richard J. Brand', 'Jerome Y. Lettvin', 'Thomas R. Fleming', 'Joseph D. Mosca', 'Erin Pleasance', 'Nahida H. Gordon', 'Calvin B. Harley', 'Russell Higuchi', 'Réka Albert', 'Keaven M. Anderson', 'J. F. Forbes', 'M Fernandez', 'Zhiping Gu', 'Bala Swaminathan', 'Jeffrey Vieira', 'Annie Yang', 'P. B. Hirsch', 'R. S. J. Frackowiak', 'Tavia Gordon', 'Ralf W. Grosse-Kunstleve', 'James R. Rice', 'V. S. Kalyanaraman', 'Daniel C. Koboldt', 'James B. Hicks', 'Jerome S. Bruner', 'Elia Stupka', 'Michael C. Jensen', 'Randi Y. Leavitt', 'Stephen F. Altschul', 'Hidemi Watanabe', 'Andrey Sivachenko', 'William J. Rutter', 'Michael Fiorino', 'Louis Fehrenbacher', 'James J. Gibson', 'Béatrice Vanbervliet', 'Richard C. Eastman', 'Qui-Lim Choo', 'Rodger Staden', 'Jeffrey C. Barrett', 'Donald B. Rubin', 'Fredrik Ronquist', 'Fuli Yu', 'Reubin Andres', 'Jacob Cohen', 'Joseph H. Roe', 'Troy Moore', 'David Tilman', 'Zhongwu Lai', 'Jonathan Sebat', 'Wonhee Jang', 'David W. Bilheimer', 'P. Armitage', 'Kalevi Pyörälä', 'John E. Ware', 'J. J. Hopfield', 'Norman E. Breslow', 'Julius Marmur', 'J. D. A. Van Embden', 'Christoph Kalka', 'Eric S. Lander', 'George L. Gabor Miklos', 'Douglas B. Rusch', 'Katherine S. Elliott', 'Alan Coulson', 'Jeffrey N. Rottman', 'Scott V. Tingey', 'Arne Tiselius', 'Dennis A. Gilbert', 'Wendy Winckler', 'Sean Humphray', 'Max Halperin', 'E. Wolf', 'Douglas L. Johnson', 'R. C. Turner', 'Arthur M. Sobol', 'Cladd E. Stevens', 'S. Cobbe', 'E. P. Greenberg', 'Howard H. Hiatt', 'P. J. Randle', 'E. Orowan', 'Aaron T. Beck', 'M Klagsbrun', 'Yves-Alain Barde', 'Daniel Kahneman', 'Robert T. DeBoy', 'G. E. Palade', 'Adam Frankish', 'Sam T. Roweis', 'Andersen Kw', 'Thomas J. Dull', 'Michaele C. Christian', 'David R. Cox', 'Donald Geman', 'M. H. F. Wilkins', 'William J. Henzel', 'Brygg Ullmer', 'Carol Redmond', 'Michael B. Eisen', 'Chang Gong Liu', 'Paul Milgrom', 'Trampisch Hj', 'M. D. Poulik', 'Daniel W. Bradley', 'Carl-Magnus Rudenstam', 'Albert László Barabási', 'David S. Hogness', 'Michael L. Metzker', 'Donald Pearson', 'J. S. Turner', 'James F. Fries', 'Suzannah Bumpstead', 'Ben Z. Locke', 'Marianne Kearney', 'R. Mead', 'Tim Hubbard', 'William Saurin', 'R. R. Ernst', 'Federico Girosi', 'Peter J. Sabo', 'Alex Bateman', 'Dienstag Jl', 'Ed Harlow', 'Carrie Sougnez', 'Guy Plunkett', 'Richard Bellman', 'Aaron McKenna', 'Gary J. Olsen', 'Lawrence L. Kupper', 'Mark D. Adams', 'Luther T. Clark', 'Janet M. Corrigan', 'A. F. Huxley', 'Nigel P. Carter', 'D Wright', 'William H. Stein', 'Robert S. Fulton', 'Heidi Greulich', 'Sankar Swaminathan', 'Toby J. Gibson', 'Simon G. Gregory', 'Aravind Subramanian', 'C. Katz', 'Tetsuo Maita', 'Helena Hämäläinen', 'Eugene W. Myers', 'Rhian Gwilliam', 'Anup Madan', 'Forrest E. Kendall', 'Cynthia L. Ogden', 'Aileen Robertson', 'Harold T. Dodge', 'Marilyn Bergner', 'D. R. Cox', 'G. I. Taylor', 'Jonathan Butler', 'Andrew B. Lassar', 'S Mason', 'Roland M. H. Schein', 'Margo A. Denke', 'Richard M. Goldstein', 'Robert B. Catalano', 'Alexander E. Urban', 'Harvey J. Alter', 'Monique G. Lê', 'Gyorgy Hutvagner', 'Darren A. Natale', 'Stanley J. Korsmeyer', 'J. Craig Venter', 'D Nelson', 'Richard Tizard', 'W Jack', 'Lee Rowen', 'Hailing Hsu', 'Robert P. Gaynes', 'Donald R. Huffman', 'Richard H. Creech', 'Michael H. Kutner', 'Michael A. Saunders', 'R Peto', 'Carol J. Bult', 'Ian Korf', 'D. R. Curtis', 'Michael Held', 'Michael R. Hee', 'Ali A. Mokdad', 'Frank Delaglio', 'Donald S. Fredrickson', 'Jeffrey S. Beck', 'Shirley Zhu', 'Jorge J. Yunis', 'Richard A. Olshen', 'David Altshuler', 'Gabriele Bergers', 'Sandra W. Clifton', 'Colin N. Dewey', 'Georges Köhler', 'Paul T. Spellman', 'Stanley Lemeshow', 'Dominic A. Scudiero', 'Basil M. Rifkind', 'Walter M. Fitch', 'Endel Tulving', 'J. M. Bland', 'Nikola P. Pavletich', 'Nawab Qizilbash', 'Raymond J. MacDonald', 'W. H. Zachariasen', 'Daniel T. Colbert', 'D. R. Matthews', 'Patrick Brill-Edwards', 'Michael K. Showe', 'Marc van de Wetering', 'Bela Julesz', 'Earl Hubbell', 'Scott L. Zeger', 'Peter K. Wellauer', 'Zbyszek Otwinowski', 'Charles T. Kresge', 'Helen Van Vunakis', 'Nicoletta Sacchi', 'Douglas G. Altman', 'R Doll', 'Richard W. Reynolds', 'A. Klug', 'Amos Tversky', 'Quinn L. Deveraux', 'Patrick O. Brown', 'Patrick D. Wall', 'David Botstein', 'K Matsushima', 'Brufman G', 'Zhaodan Cao', 'Elisabeth Buchdunger', 'Alex Pentland', 'F. P. Bowden', 'J Holmes', 'Kenneth J. Livak', 'K Sakai', 'Hoda Khouri', 'Jerzy Neyman', 'R. Willox', 'Raymond G. Goodwin', 'Richard Durbin', 'Alan J. Buckler', 'Fu Lu', 'Gordon J', 'Pablo Tamayo', 'Shimon Ullman', 'Gary W. Trucks', 'Morris J. Karnovsky', 'J. S. Waugh', 'Wen Lin Kuo', 'Nicholas B. Lydon', 'Amy J. Weiner', 'Anne S. Olsen', 'Allan C. Wilson', 'Daniel H. Haft', 'Stephen E. Haynesworth', 'Marek Kubin', 'Carol Churcher', 'Todd D. Taylor', 'Jane Rogers', 'Teresa C. Horan', 'S. Korzeniowski', 'Nick Goldman', 'John P. Huelsenbeck', 'Huyen Dinh', 'Maynard V. Olson', 'Vern Paxson', 'Joel E. Richardson', 'John C. Cassel', 'Josep F. Abril', 'Robert F. Kibler', 'H. Gervásio', 'David I. Gottlieb', 'Sarah Edkins', 'Howard K. Schachman', 'David Beach', 'Andrew P. Feinberg', 'Hilary King', 'John C. Doyle', 'Jon Teague', 'Laurence Kennedy', 'Anthony R. Kerlavage', 'Drummond Rennie', 'Allyson Ross Davies', 'George Kuo', 'Jian Bing Fan', 'Gwyneth Van', 'Paul McGale', 'Ellson Y. Chen', 'Trevor Hastie', 'Pierre R. Villeneuve', 'Thomas Tuschl', 'Richard R. Copley', 'Torben Palshof', 'James R. Hudson', 'Sandra Silberman', 'John McCarthy', 'Wieslaw J. Roth', 'Douglas J. Schwartzentruber', 'Ian Dunham', 'Carol W. Greider', 'Leslie L. Iversen', 'Sean R. Eddy', 'Claudia A. Seipp', 'Frederic M. Richards', 'Lawrence G. Hunsicker', 'Rajinder Kaul', 'Mark R. Chassin', 'Susan E. Folstein', 'Richard Breathnach', 'Thomas R. Dawber', 'E. A. McCulloch', 'Stephen C. Reingold', 'Mukesh Dherani', 'F Taylor', 'Santos A. Susin', 'Simon Myers', 'Ronald H. Reeder', 'Hilary A. Coller', 'James E. Grizzle', 'Lynn A. G. Ries', 'Eugene L. Brown', 'Robert Karasek', 'Edward Witten', 'Myron Flickner', 'Oliver H. Lowry', 'A. L. Hodgkin', 'Kenneth J. Arrow', 'Jorja G. Henikoff', 'G. E. Hutchinson', 'Nancy A. Thornberry', 'Naoufal Zamzami', 'A. P. Alivisatos', 'Vincent P. Stanton', 'E Thomson', 'Deanna M. Church', 'Mark S. Chee', 'Joe M. McCord', 'François Artiguenave', 'Liisa Holm', 'Eladio Viñuela', 'Mary Burden', 'Granger G. Sutton', 'Oliver Smithies', 'Herbert W. Boyer', 'William H. Habig', 'J Graham', 'J. R. Vane', 'Therese Sørlie', 'Audrey Duncanson', 'Marjorie B. Lees', 'Franklin Kim', 'Maurizio Belfiglio', 'C Davies', 'Edmund A. Gehan', 'Jill P. Mesirov', 'Alastair Kent', 'John R. Glover', 'Susan M. Hailpern', 'Patrick Therasse', 'Sarah Hunt', 'Arthur L. Delcher', 'Gavin Sherlock', 'Martin Gibson', 'Robert P. Abelson', 'Kristin Ardlie', 'Timothy B. Stockwell', 'Zhengdong D. Zhang', 'Stewart Craig', 'Charles Dauguet', 'Thomas Jenuwein', 'Dean Burk', 'Joseph Ragaz', 'Renaud Capdeville', 'Halsted R. Holman', 'Mark H. Skolnick', 'Harold Swerdlow', 'Jennifer Baldwin', 'Roger E. Kirk', 'Erik L. L. Sonnhammer', 'Adam Auton', 'Abraham Kagan', 'Marianne Sullivan', 'Torsten N. Wiesel', 'Vishwanath R. Iyer', 'W. Brian Arthur', 'Michael L. Watson', 'Darrel A. Regier', 'Graeme A. Reid', 'D. H. Auston', 'Srinivasa M. Srinivasula', 'Daniel I. Arnon', 'J Clarke', 'Marshal F. Folstein', 'Tullia Lindsten', 'Robert J. Lipshutz', 'Lucy Matthews', 'Edwin M. Southern', 'William B. Kannel', 'James C. Mullikin', 'Matthew Stephens', 'Jane Grimwood', 'Ross L. Prentice', 'Laura Elnitski', 'Alexandre Montpetit', 'M. G. Bawendi', 'Nathan O. Kaplan', 'Jim Gray', 'Elizabeth Lacy', 'Mark Yandell', 'Christopher B. Burge', 'Andrew Kasarskis', 'Alan T. Bankier', 'Clifford L. Johnson', 'Koichiro Tamura', 'Harold Weintraub', 'L. Di Martino', 'Robert A. Weinberg', 'Susan L. Naylor', 'Andrew Wakeham', 'Karin Eiglmeier', 'John G. Kirkwood', 'Scott Uknes', 'Steven E. Scherer', 'Jacques Ferlay', 'David L. Donoho', 'Timothy Fennell', 'Genji Matsuda', 'Barbara E. Stranger', 'Eric A. Jaffe', 'Daphne W. Bell', 'D. Williams Parsons', 'John V. Bennett', 'Matt van de Rijn', 'Parkhurst A. Shore', 'Paul Liska', 'Aram V. Chobanian', 'James J. Heckman', 'Simon L. Howell', 'R. W. Butcher', 'S. M. Zahurak', 'Pamela Whittaker', 'Robert A. Rosati', 'Sarah Nutland', 'David Torrents', 'Yoav Freund', 'Gabor T. Marth', 'Catherine A. Ball', 'T. N. Bhat', 'Mark J. Daly', 'Eric A. Swanson', 'Scott L. Weinrich', 'Robert Katzman', 'Taylor A. Sohn', 'Erding Hu', 'Andreas Thess', 'Sj Wilson', 'Ants Leetmaa', 'P. F. Scholander', 'Allen Newell', 'Argiris Efstratiadis', 'John P. Feighner', 'Lisa French', 'Larry Rubinstein', 'A. L. Page', 'Catherine Michaud', 'Jean Endicott', 'Jacques Monod', 'Anne Dornhorst', 'Kara Dolinski', 'Stanley L. Wallace', 'Kurt J. Greenlund', 'Emily Bernstein', 'Francine Brière', 'Ronald Melzack', 'Gerald M. Rubin', 'Richard Helm', 'C Fox', 'Robert G. Parr', 'David B. Johnson', 'Lon R. Cardon', 'Gail R. Martin', 'David L. Swofford', 'Roland Heilig', 'Helen Goodell', 'John H. Holland', 'Francis M. Miezin', 'Frederick Mosteller', 'J.Ward Kennedy', 'Sidney J. Cutler', 'Michael S. Gottlieb', 'John R. Marler', 'Keith J. Worsley', 'Waldo E. Cohn', 'Joan Keesey', 'Andrew Y. Ng', 'Brian Haynes', 'U. Veronesi', 'Warren J. Hehre', 'Allan Van Oosterom', 'Martin Fishbein', 'Arian Smit', 'James B. Wyngaarden', 'Robin Humphry-Baker', 'David G. Lowe', 'Charles D. Keeling', 'Alan D. Lopez', 'Kimmen Sjölander', 'David G. Kleinbaum', 'W. James Kent', 'J Martin Bland', 'Robert J. Dodson', 'K. S. Novoselov', 'Mercedes Camalli', 'Alexej Abyzov', 'Melvin P. Judkins', 'William H. Majoros', 'D. Lawrence Wickerham', 'Paul Doty', 'Ewan Birney', 'Daniel W. Jones', 'Kung-Yee Liang', 'Gordon Saksena', 'Solange Chamaret', 'Frederick C. Neidhardt', 'John Nash', 'Michael S. Waterman', 'Thomas N. Sato', 'M. Castiglione', 'Geoffrey E. Hinton', 'Wolfgang Kabsch', 'David Julius', 'David L. Pearle', 'Dennis Y. Loh', 'Ronald A. Laskey', 'Thomas C. Chalmers', 'Noriko Sakaguchi', 'Eric Pelletier', 'Jeffrey A. Cutler', 'Thomas J. Palker', 'Michael D. McLellan', 'M. G. Sarngadharan', 'Michael Feolo', 'Taylor Ja', 'Valerie A. Fadok', 'Andreas Rump', 'F. C. Greenwood', 'Vladimir Korinek', 'Reto Ruedy', 'Lore Zech', 'A. C. T. North', 'Peter W. J. Rigby', 'Donald P. Nierlich', 'Edgardo Lander', 'Jeremy D. Glasner', 'Sarah S. Murray', 'Rebecca DerSimonian', 'Clifton F. Mountain', 'David R. Bentley', 'R. A. Fisher', 'Andrew Fire', 'Gordon L. Shulman', 'Anita L. Stewart', 'Ahmedin Jemal', 'Desmond G. Higgins', 'John M. Darley', 'H. J. Stewart', 'Isaac L. Chuang', 'David J. Dooling', 'David W. Tank', 'Yuval Eshed', 'Monica Williams', 'Kikuko Hotta', 'Nj Bell', 'Kazuto Kato', 'Linus Pauling', 'Freddie Bray', 'Bert Vogelstein', 'K McPherson', 'Werner Risau', 'Evan Mauceli', 'Federica Sallusto', 'Charles Atkinson', 'Paul Emsley', 'D. C. Van Essen', 'P Park', 'John D. Lafferty', 'Donna F. Stroup', 'Mark Weiser', 'Charles H. Hennekens', 'Beverley Shields', 'Hugh Paterson', 'Ken Dewar', 'John E. Helzer', 'Sanford L. Palay', 'George Paxinos', 'Liesi E. Hebert', 'Robert C. Onofrio', 'Rachel M. Freathy', 'David A. Hungerford', 'Yasunori Ogura', 'Bruce N. Ames', 'Sam Griffiths-Jones', 'Catrina Fronick', 'Ivan Lieberburg', 'Stephen T. Sherry', 'Charmaine D. Royal', 'N. F. Mott', 'Douglas T. Ross', 'H. Murayama', 'Marshall N. Rosenbluth', 'Rodney M. Hewick', 'W. M. Cowan', 'Karen Hsiao', 'Timo T. Valle', 'Yasushi Totoki', 'Amos Marc Bairoch', 'William A. Knaus', 'William J. Fulkerson', 'Christopher K. Raymond', 'Ross A. Lippert', 'John A. Pople', 'W. Richard McCombie', 'Lora Lewis', 'Michael W. Hunkapiller', 'B. Gutenberg', 'Karen Eilbeck', 'John R. Carson', 'Jeremy White', 'Shelley E. Taylor', 'Tarjei S. Mikkelsen', 'Vincent Ferretti', 'Thomas H. Holmes', 'Karsten Hokamp', 'David Haussler', 'Graham R. Bignell', 'W. Dansgaard', 'Paul Ekman', 'Amy A. Caudy', 'Ingrid Daubechies', 'B. A. Roe', 'A. Howie', 'Joachim Messing', 'Franz Josef Och', 'Eli Robins', 'Bates Dw', 'H. R. Horvitz', 'Michael Ashburner', 'Hamilton O. Smith', 'William N. Anderson', 'Kenneth W. Kinzler', 'Joseph H. Connell', 'Tom Maniatis', 'John M.C. Gutteridge', 'Suzanna E. Lewis', 'Rolf Apweiler', 'Robert I. Levy', 'Paul J. Deutsch', 'Eric W. Wong', 'Robin Milner', 'Keith R. Porter', 'Stefan Taudien', 'S Takashima', 'Michael J. Thun', 'Joe Sambrook', 'E. A. Clarke', 'David W. Leung', 'Marco Punta', 'M. Karplus', 'B. F. Skinner', 'Vishva M. Dixit', 'Andrew G. Fraser', 'Brian R. Lawn', 'Max H. Myers', 'Sewall Wright', 'Martine Van Glabbeke', 'Bruce Culleton', 'Martin L. Hooper', 'Nancy F. Hansen', 'Albert J. Belanger', 'Kirk R. Thomas', 'Andrew Menzies', 'Stanford Moore', 'Miller Ja', 'Max Born', 'Ulrich K. Laemmli', 'Walter Gilbert', 'Tracy M. Bryan', 'G. Flandrin', 'Alfonse T. Masi', 'Giorgio Cocconi', 'James H. Stock', 'Nathan Mantel', 'Kenneth P. Burnham', 'Lawrence E. Shulman', 'Sheila Seal', 'David S. Johnson', 'Gonçalo R. Abecasis', 'Donald E. Groom', 'Ramzi S. Cotran', 'S Clarke', 'Owen P. Hamill', 'A. K. Geim', 'William P. Castelli', 'Jinghui Zhang', 'Folke Skoog', 'Alejandro A. Schäffer', 'George E. Palade', 'Paul E. Meehl', 'Lee Murphy', 'Antman Em', 'René O. Mirimanoff', 'Bonnee Rubinfeld', 'Jennifer J. Anderson', 'Gitanjali M. Singh', 'Anthony J. Cox', 'Richard O. Russell', 'David L. Sackett', 'Felsenfeld Al', 'Jane Peterson', 'H. Craig', 'M Schumacher', 'Sergey V. Morozov', 'Jacqueline Kosecoff', 'P. J. Flory', 'Irving Langmuir', 'Albert H. Coons', 'Roel P. Funke', 'Steven L. Salzberg', 'Alane Gray', 'James W. Curran', 'Lars Hedin', 'G. A. R. Johnston', 'G. Bonadonna', 'Auke Tellegen', 'Michael R. Brent', 'al. et', 'Albert-László Barabási', 'Earl W. Sutherland', 'William Haenszel', 'Dmitriy A. Dikin', 'J. P. Grime', 'N. Ferrara', 'A. D. Bangham', 'Karen A. Ketchum', 'Sidney C. Smith', 'Douglas P. Wagner', 'Don W. Fawcett', 'Steven H. Strogatz', 'Jonathan K. Pritchard', 'Toby Bloom', 'Deborah R. Shapiro', 'Phyllis A. Wingo', 'Ronald L. Rivest', 'I. V. Grigorieva', 'Mary Osborn', 'Glenn Tesler', 'P. J. Sellers', 'F. Sanger', 'David H. Gelfand', 'Howard A. Eder', 'J. L. Weiland', 'Jennifer R. Wortman', 'Daniel McGee', 'G. D. Farquhar', 'John F. Heidelberg', 'Yuen Shing', 'Brenda K. Edwards', 'John Bowlby', 'Ruslan Medzhitov', 'Ching W. Tang', 'Norman A. Doggett', 'Gerald R. Fink', 'Trevor Hawkins', 'Samuel B. Guze', 'Richard R. Hudson', 'Solomon A. Berson', 'Eugene R. Passamani', 'David B. Jaffe', 'Roger Y. Tsien', 'Bigger Jt', 'Victor R. Ambros', 'David H. Culver', 'Donald D. Van Slyke', 'John E. Wennberg', 'McGuire Wl', 'Wayne J. Higgins', 'Jingtao Sun', 'Reginald A. Clift', 'Jitendra Malik', 'Gordon G. Wong', 'Fred Ramsdell', 'Heather Schmidt', 'J. S. Bell', 'P. G. Smith', 'Michael L. Frigge', 'Gilbert Chu', 'Irene M. Stratton', 'Martin Ringwald', 'Katherine W. Timothy', 'William H. Dietz', 'Joseph L. Fleiss', 'E Howard', 'Yuri Lazebnik', 'Cyrus Levinthal', 'Stephen Frank Schaffner', 'M. A. Fiatarone', 'Cole Trapnell', 'J. Milburn Jessup', 'Chris C A Spencer', 'Richard M. Karp', 'Henry A. Erlich', 'Jörg Schultz', 'Gary Struhl', 'Steven R. Gill', 'Patrick Minx', 'Craig H. Mermel', 'L. J. Ignarro', 'Kevin McKernan', 'Ahmed M. El-Nahas', 'Joseph O. Hirschfelder', 'C. Pennypacker', 'Aaron L. Halpern', 'Pere Puigserver', 'James G. Hirsch', 'J. S. Binkley', 'William R. McCabe', 'Andrew H. Wyllie', 'Morton Kramer', 'Alexander Parkhomenko', 'Kenneth Macleod', 'Cyrus Chothia', 'A. H.-J. Wang', 'Helmut Blöcker', 'Tong Ihn Lee', 'Hans-Gerd Lipinski', 'Mary Jane Ferraro', 'Michael Zuker', 'D. C. Wiley', 'Marc Zabeau', 'Jeremy Peterson', 'Henderson Ic', 'Insa Gathmann', 'Patrice J. Morin', 'D. Easton', 'F. Barre-Sinoussi', 'Rodney A. Brooks', 'P.-G. De Gennes', 'Jean Morissette', 'David J. Lockhart', 'Betty J. Flehinger', 'Robert H. Waterston', 'Karl A. Piez', 'Morten Kjeldgaard', 'C Rose', 'Michael E. Greenberg', 'Kazuhiko Kawasaki', 'Marius Wernig', 'Margherita Maffei', 'William J. Martone', 'Kiran Garimella', 'A. Naja', 'Jane G. Schaller', 'Ralph S. Paffenbarger', 'Masatoshi Nei', 'Randall K. Saiki', 'Michael Polanyi', 'Carl R. Woese', 'Nicola Mulder', 'Ilya Spigland', 'Stephen F. Jencks', 'Michael C. Wendl', 'Robert L. Spitzer', 'Marcy Silver', 'Marcus E. Raichle', 'George Scatchard', 'Leonid Kruglyak', 'Lynn R. Sykes', 'Ali Zandieh', 'Kristian Cibulskis', 'Peter Philippsen', 'Joseph Sambrook', 'Robert Kamen', 'James I. Cleeman', 'Hiroaki Hemmi', 'Lucinda Fulton', 'Carole A. Cull', 'David W. Hosmer', 'P. Andrew Futreal', 'Edgar F. Meyer', 'T. J. Sejnowski', 'Paul W. Gast', 'John C. Khoo', 'Lee N. Robins', 'Webb Miller', 'Rory Collins', 'Chad Nusbaum', 'Lynn M. Schriml', 'Martin Poenie', 'Joseph Berkson', 'Eric R. Fearon', 'Raymond D. Miller', 'Sidney P. Colowick', 'Jay D. Pearson', 'Marie-Angèle Morel']

WITH a, COUNT(q) AS num_top_collabs
WHERE num_top_collabs > 0

MATCH (a)-[:AUTHORED]->(q:Quanta)<-[:AUTHORED]-(b:Top42Author)
    WHERE q.venue in ['Angewandte Chemie','Blood','Cancer Cell','Cancer Discovery','Cancer Research','Cell','Cell Host & Microbe','Cell Metabolism','Cell Stem Cell','Chemistry & Biology','The EMBO Journal','Genes & Development','Immunity','Journal of Neurology','Journal of the American Chemical Society','JAMA','Journal of Biological Chemistry','Journal of Cell Biology','Journal of Clinical Investigation','Journal of Experimental Medicine','Journal of Medicinal Chemistry','The Lancet','Nature Cell Biology','Nature Chemical Biology','Nature Chemistry','Nature Medicine','Nature Methods','Nature','Nature Biotechnology','The New England Journal of Medicine','Neuron','Nature Genetics','Nature Immunology','Nature Neuroscience','Nature Structural & Molecular Biology','PLOS Biology','PLOS Genetics','PLOS Pathogens','Proceedings of the National Academy of Sciences of the United States of America','Science Signaling','Science Translational Medicine','Science']
         AND a.last_author_yr <= q.year AND b.last_author_yr <= q.year  
    WITH a, b, 
        (toFloat(1) / (11.327052 + avg(q.num_profs_v2))) as u_lin,
        (toFloat(1) / sqrt(11.327052 + avg(q.num_profs_v2))) as u_sqrt,
        (toFloat(1) / (11.327052 + avg(q.num_profs_v2))^2) as u_square,
        (toFloat(1) / exp(11.327052 + avg(q.num_profs_v2))) as u_exp
        
    RETURN a.name as name, 
        a.last_author_yr as last_author_yr,
        count(u_lin) as unique_coauthor_profs, 
        11.327052*sum(u_lin) as unique_coauthor_profs_lin_damp,
        sqrt(11.327052)*sum(u_sqrt) as unique_coauthor_profs_sqrt_damp,
        11.327052^2*sum(u_square) as unique_coauthor_profs_square_damp,
        exp(11.327052)*sum(u_exp) as unique_coauthor_profs_exp_damp"
        , "/data/csv/Collab_Early_Profs_3.csv", {batchSize:100, iterateList:true, parallel:true})
    """
df_collab = query_to_df(query, graph)
# df_collab.to_csv('~/Workspace/UROP/scaling-science/notebooks/Collaboration/Data/Collab_Early_Profs.csv', index = False, encoding = "UTF-8")

Starting query... Done (0.20 minutes).


In [28]:
# Impact
query = """
call apoc.export.csv.query(
"MATCH (a:Top42Author)
WHERE a.last_author_yr > 2013 

MATCH (a)-[:AUTHORED]->(q:Quanta)<-[:AUTHORED]-(b:Author)
WHERE q.venue in ['Angewandte Chemie','Blood','Cancer Cell','Cancer Discovery','Cancer Research','Cell','Cell Host & Microbe','Cell Metabolism','Cell Stem Cell','Chemistry & Biology','The EMBO Journal','Genes & Development','Immunity','Journal of Neurology','Journal of the American Chemical Society','JAMA','Journal of Biological Chemistry','Journal of Cell Biology','Journal of Clinical Investigation','Journal of Experimental Medicine','Journal of Medicinal Chemistry','The Lancet','Nature Cell Biology','Nature Chemical Biology','Nature Chemistry','Nature Medicine','Nature Methods','Nature','Nature Biotechnology','The New England Journal of Medicine','Neuron','Nature Genetics','Nature Immunology','Nature Neuroscience','Nature Structural & Molecular Biology','PLOS Biology','PLOS Genetics','PLOS Pathogens','Proceedings of the National Academy of Sciences of the United States of America','Science Signaling','Science Translational Medicine','Science']
    AND q.year < a.last_author_yr
    AND b.name in ['David P. Reed', 'Mikulas Popovic', 'Katherine M. Flegal', 'Boris Reva', 'Emily S. Winn-Deen', 'B. G. Barrell', 'Richard G. Margolese', 'John M. Chirgwin', 'Floyd Ratliff', 'Joan Massagué', 'Sarah Milne', 'M. Overgaard', 'Joseph L. Goldstein', 'Todd M. Lowe', 'Jonathan Marchini', 'Peter J. Ramadge', 'Daniel R. Marshak', 'Stephen M. J. Searle', 'Leon Festinger', 'James Kennedy', 'Michael B. Mock', 'Juha Kononen', 'Julia B. Lewis', 'Jonathan R. Edwards', 'Victor E. Velculescu', 'Peter J. Turnbaugh', 'Akira Nagafuchi', 'Amy E. Pasquinelli', 'Norman Arnheim', 'Thomas D. Schmittgen', 'Linda M. Rolnitzky', 'Rodrigo Lopez', 'J.-C. Gazet', 'Joel S. Hayflick', 'P. C. Eklund', 'Stefanie S. Jeffrey', 'Camille Parmesan', 'N. Cabrera', 'Andrew J. Mungall', 'Robert Baertsch', 'Robert E. Bruccoleri', 'Robert S. Pinals', 'Helen Skaletsky', 'Keith Nelson', 'J. Richard Hackman', 'J. Michael Cherry', 'Mark F. Pittenger', 'Richa Agarwala', 'Mark A. DePristo', 'James H. Ware', 'Serafim Batzoglou', 'Lee A. Green', 'J. E. Till', 'Elizabeth Ward', 'Sheila Scully', 'William T. Friedewald', 'Allan M. Skene', 'V. Narry Kim', 'Tina Graves', 'Elaine R. Mardis', 'George W. Torrance', 'Robert L. Coffman', 'Andreas Heger', 'Ellen E. Walters', 'Jeremy D. Selengut', 'Ruth E. Ley', 'C. V. Shank', 'Gregory J. Hannon', 'Donald S. Rae', 'James A. Stockman', 'Saira Salahuddin', 'J. G. Bednorz', 'D. von Fournier', 'J. F. A. P. Miller', 'Scott Schwartz', 'Ivica Letunic', 'Heike Fiegler', 'J. Gillis', 'Russell Ross', 'Christie Kovar', 'Bruce W. Birren', 'Richard S. Lazarus', 'Helga Boedtker', 'George A. Miller', 'George H. Mead', 'Arthur Purdy Stout', 'LaDeana W. Hillier', 'William A. Sibley', 'Michael P. Bevilacqua', 'Michele L. Pearson', 'Jean-Roger Le Gall', 'M. R. Cooper', 'Jean Thierry-Mieg', 'Joanna E. Siegel', 'Raffaella Sordella', 'David J. Spiegelhalter', 'Frans Bruyninckx', 'Bruce M. Spiegelman', 'George G. J. M. Kuiper', 'Stacey Gabriel', 'Seymour S. Kety', 'Kenneth F. Schulz', 'S. L. McCall', 'Morris W. Foster', 'Huaiyu Mi', 'Nick Patterson', 'Donna K. Slonim', 'Richard H. Chapman', 'Clare Seamark', 'Saul A. Teukolsky', 'Craig C. Mello', 'E Robinson', 'A E Weyman', 'Hartmut Land', 'Ramon Parsons', 'Katsuaki Hoshino', 'George J. Taylor', 'Deborah Wentworth', 'M Richard', 'Thomas E. Carew', 'David J. Lipman', 'Erich Bluhmki', 'John D. McPherson', 'Robert J. Garrison', 'William F. Novotny', 'Kary B. Mullis', 'Robert M. Weisskoff', 'Zemin Ning', 'Daniel Shoemaker', 'M Bond', 'Jillian Boreham', 'Zoltán N. Oltvai', 'Jay R. Ritter', 'Leland H. Hartwell', 'G. Binnig', 'Tomaso Poggio', 'Nathan Maccoby', 'Donald D. Newmeyer', 'Benjamin A. Barnes', 'Charles Tanford', 'M. Michele Manos', 'Otto A. Bessey', 'Yudong D. He', 'Herbert A. Simon', 'Deborah Estrin', 'John V. Moran', 'Gabor Gyapay', 'Michael Shi', 'Edward J. Roccella', 'Richard D. Piner', 'Taro Kawai', 'Erica Sodergren', 'Stephen J. Aldington', 'M. Y. Kim', 'Gilean McVean', 'John P. Perdew', 'Scott Shenker', 'S. Moncada', 'Anders Krogh', 'Jerome Cornfield', 'Ralph M. Steinman', 'M. E. J. Newman', 'Jeffrey C. Lee', 'Benjamin D. Hall', 'Miriam J. Alter', 'Sidney Udenfriend', 'James K. V. Willson', 'Mark S. Guyer', 'Julian Peto', 'James E. Galagan', 'Alice C. Young', 'Theophil Staehelin', 'Steven D. Averbuch', 'Edmund G. Lowrie', 'Cliff Tabin', 'P Sandercock', 'Nathan Goodman', 'Richard A. Shafer', 'Steve Rozen', 'A. R. Hunt', 'J.A. van Dongen', 'Mary C. Betlach', 'Michael C. Zody', 'Harry Towbin', 'Myunghee C. Paik', 'Mark Diekhans', 'Michael Gent', 'Mark S. Boguski', 'Hongjie Dai', 'David V. Goeddel', 'Andrew Neil', 'Yunzhen Cao', 'M. H. Gault', 'Paul Meier', 'Robert E. Kleiger', 'Kim Rutherford', 'William R. Pearson', 'Janan T. Eppig', 'Bert Sakmann', 'Stuart McLaren', 'R Parker', 'James H. Jandl', 'David P. Bartel', 'C Redmond', 'Jerome Vinograd', 'Mark Dickson', 'Barry A. Morgan', 'Julie D. Thompson', 'Mike Rothe', 'Tobias Doerks', 'Jeffrey G. Reid', 'D. J. E. Ingram', 'Yuling Hong', 'Barbara J. McNeil', 'Robert Tibshirani', 'Jeremy H. Burroughes', 'Ueli Gubler', 'C. Milstein', 'Paul J. Flory', 'A. K. Mitchell', 'Mark T. Ross', 'I. M. Stratton', 'P. D. Home', 'Andrew P. Holmes', 'C. Arden Pope', 'Tim R. Mosmann', 'Richard Peto', 'Richard C. Graham', 'John M. Leonard', 'Arthur B. Lamb', 'Benoit B. Mandelbrot', 'David E. Goldberg', 'E. Kellenberger', 'F. Schedin', 'C. K. Osborne', 'Nicholas Kassebaum', 'Ch. Gerber', 'Russell Turner', 'Stéphane Guindon', 'Frank E. Speizer', 'Susan G. Arbuck', 'Thomas D. Willis', 'R W Davis', 'Phillip D. Zamore', 'Nan M. Laird', 'Barth Hoogstraten', 'Kim C. Worley', 'Hugo Muench', 'Douglas Hanahan', 'Margaret D. Carroll', 'Gary Ruvkun', 'B. E. Warren', 'Catherine A. Lozupone', 'Robert F. Meenan', 'Naruya Saitou', 'James S. Marks', 'Wilson Ia', 'Minze Stuiver', 'Seymour Benzer', 'Terrence S. Furey', 'Judah Folkman', 'Richard E. Nisbett', 'Pui Yan Kwok', 'Mark A. Saper', 'Erika Schmid', 'Marita Cross', 'Dennis G. Ballinger', 'Elisabeth Burdick', 'D. Wayne Taylor', 'Michele Cargill', 'Fred Ederer', 'Loren F. Hiratzka', 'Oliver Schwardt', 'Saul Genuth', 'Stuart Bunting', 'Arnold Oliphant', 'N. E. Day', 'Paul Predki', 'Paul D. Sorlie', 'Donna M. Rennick', 'Louis A. Tartaglia', 'Parnesh Raniga', 'Robert Axelrod', 'A. McGuire', 'Stewart Scherer', 'Lucian L. Leape', 'U. J. Lewis', 'J Burns', 'Sarah E. Fowler', 'Bernard B. Brodie', 'Ghia Euskirchen', 'Kate Downes', 'Scott M. Hammond', 'Robert D. Fleischmann', 'Robert M. Kuhn', 'Richard J. Brand', 'Jerome Y. Lettvin', 'Thomas R. Fleming', 'Joseph D. Mosca', 'Erin Pleasance', 'Nahida H. Gordon', 'Calvin B. Harley', 'Russell Higuchi', 'Réka Albert', 'Keaven M. Anderson', 'J. F. Forbes', 'M Fernandez', 'Zhiping Gu', 'Bala Swaminathan', 'Jeffrey Vieira', 'Annie Yang', 'P. B. Hirsch', 'R. S. J. Frackowiak', 'Tavia Gordon', 'Ralf W. Grosse-Kunstleve', 'James R. Rice', 'V. S. Kalyanaraman', 'Daniel C. Koboldt', 'James B. Hicks', 'Jerome S. Bruner', 'Elia Stupka', 'Michael C. Jensen', 'Randi Y. Leavitt', 'Stephen F. Altschul', 'Hidemi Watanabe', 'Andrey Sivachenko', 'William J. Rutter', 'Michael Fiorino', 'Louis Fehrenbacher', 'James J. Gibson', 'Béatrice Vanbervliet', 'Richard C. Eastman', 'Qui-Lim Choo', 'Rodger Staden', 'Jeffrey C. Barrett', 'Donald B. Rubin', 'Fredrik Ronquist', 'Fuli Yu', 'Reubin Andres', 'Jacob Cohen', 'Joseph H. Roe', 'Troy Moore', 'David Tilman', 'Zhongwu Lai', 'Jonathan Sebat', 'Wonhee Jang', 'David W. Bilheimer', 'P. Armitage', 'Kalevi Pyörälä', 'John E. Ware', 'J. J. Hopfield', 'Norman E. Breslow', 'Julius Marmur', 'J. D. A. Van Embden', 'Christoph Kalka', 'Eric S. Lander', 'George L. Gabor Miklos', 'Douglas B. Rusch', 'Katherine S. Elliott', 'Alan Coulson', 'Jeffrey N. Rottman', 'Scott V. Tingey', 'Arne Tiselius', 'Dennis A. Gilbert', 'Wendy Winckler', 'Sean Humphray', 'Max Halperin', 'E. Wolf', 'Douglas L. Johnson', 'R. C. Turner', 'Arthur M. Sobol', 'Cladd E. Stevens', 'S. Cobbe', 'E. P. Greenberg', 'Howard H. Hiatt', 'P. J. Randle', 'E. Orowan', 'Aaron T. Beck', 'M Klagsbrun', 'Yves-Alain Barde', 'Daniel Kahneman', 'Robert T. DeBoy', 'G. E. Palade', 'Adam Frankish', 'Sam T. Roweis', 'Andersen Kw', 'Thomas J. Dull', 'Michaele C. Christian', 'David R. Cox', 'Donald Geman', 'M. H. F. Wilkins', 'William J. Henzel', 'Brygg Ullmer', 'Carol Redmond', 'Michael B. Eisen', 'Chang Gong Liu', 'Paul Milgrom', 'Trampisch Hj', 'M. D. Poulik', 'Daniel W. Bradley', 'Carl-Magnus Rudenstam', 'Albert László Barabási', 'David S. Hogness', 'Michael L. Metzker', 'Donald Pearson', 'J. S. Turner', 'James F. Fries', 'Suzannah Bumpstead', 'Ben Z. Locke', 'Marianne Kearney', 'R. Mead', 'Tim Hubbard', 'William Saurin', 'R. R. Ernst', 'Federico Girosi', 'Peter J. Sabo', 'Alex Bateman', 'Dienstag Jl', 'Ed Harlow', 'Carrie Sougnez', 'Guy Plunkett', 'Richard Bellman', 'Aaron McKenna', 'Gary J. Olsen', 'Lawrence L. Kupper', 'Mark D. Adams', 'Luther T. Clark', 'Janet M. Corrigan', 'A. F. Huxley', 'Nigel P. Carter', 'D Wright', 'William H. Stein', 'Robert S. Fulton', 'Heidi Greulich', 'Sankar Swaminathan', 'Toby J. Gibson', 'Simon G. Gregory', 'Aravind Subramanian', 'C. Katz', 'Tetsuo Maita', 'Helena Hämäläinen', 'Eugene W. Myers', 'Rhian Gwilliam', 'Anup Madan', 'Forrest E. Kendall', 'Cynthia L. Ogden', 'Aileen Robertson', 'Harold T. Dodge', 'Marilyn Bergner', 'D. R. Cox', 'G. I. Taylor', 'Jonathan Butler', 'Andrew B. Lassar', 'S Mason', 'Roland M. H. Schein', 'Margo A. Denke', 'Richard M. Goldstein', 'Robert B. Catalano', 'Alexander E. Urban', 'Harvey J. Alter', 'Monique G. Lê', 'Gyorgy Hutvagner', 'Darren A. Natale', 'Stanley J. Korsmeyer', 'J. Craig Venter', 'D Nelson', 'Richard Tizard', 'W Jack', 'Lee Rowen', 'Hailing Hsu', 'Robert P. Gaynes', 'Donald R. Huffman', 'Richard H. Creech', 'Michael H. Kutner', 'Michael A. Saunders', 'R Peto', 'Carol J. Bult', 'Ian Korf', 'D. R. Curtis', 'Michael Held', 'Michael R. Hee', 'Ali A. Mokdad', 'Frank Delaglio', 'Donald S. Fredrickson', 'Jeffrey S. Beck', 'Shirley Zhu', 'Jorge J. Yunis', 'Richard A. Olshen', 'David Altshuler', 'Gabriele Bergers', 'Sandra W. Clifton', 'Colin N. Dewey', 'Georges Köhler', 'Paul T. Spellman', 'Stanley Lemeshow', 'Dominic A. Scudiero', 'Basil M. Rifkind', 'Walter M. Fitch', 'Endel Tulving', 'J. M. Bland', 'Nikola P. Pavletich', 'Nawab Qizilbash', 'Raymond J. MacDonald', 'W. H. Zachariasen', 'Daniel T. Colbert', 'D. R. Matthews', 'Patrick Brill-Edwards', 'Michael K. Showe', 'Marc van de Wetering', 'Bela Julesz', 'Earl Hubbell', 'Scott L. Zeger', 'Peter K. Wellauer', 'Zbyszek Otwinowski', 'Charles T. Kresge', 'Helen Van Vunakis', 'Nicoletta Sacchi', 'Douglas G. Altman', 'R Doll', 'Richard W. Reynolds', 'A. Klug', 'Amos Tversky', 'Quinn L. Deveraux', 'Patrick O. Brown', 'Patrick D. Wall', 'David Botstein', 'K Matsushima', 'Brufman G', 'Zhaodan Cao', 'Elisabeth Buchdunger', 'Alex Pentland', 'F. P. Bowden', 'J Holmes', 'Kenneth J. Livak', 'K Sakai', 'Hoda Khouri', 'Jerzy Neyman', 'R. Willox', 'Raymond G. Goodwin', 'Richard Durbin', 'Alan J. Buckler', 'Fu Lu', 'Gordon J', 'Pablo Tamayo', 'Shimon Ullman', 'Gary W. Trucks', 'Morris J. Karnovsky', 'J. S. Waugh', 'Wen Lin Kuo', 'Nicholas B. Lydon', 'Amy J. Weiner', 'Anne S. Olsen', 'Allan C. Wilson', 'Daniel H. Haft', 'Stephen E. Haynesworth', 'Marek Kubin', 'Carol Churcher', 'Todd D. Taylor', 'Jane Rogers', 'Teresa C. Horan', 'S. Korzeniowski', 'Nick Goldman', 'John P. Huelsenbeck', 'Huyen Dinh', 'Maynard V. Olson', 'Vern Paxson', 'Joel E. Richardson', 'John C. Cassel', 'Josep F. Abril', 'Robert F. Kibler', 'H. Gervásio', 'David I. Gottlieb', 'Sarah Edkins', 'Howard K. Schachman', 'David Beach', 'Andrew P. Feinberg', 'Hilary King', 'John C. Doyle', 'Jon Teague', 'Laurence Kennedy', 'Anthony R. Kerlavage', 'Drummond Rennie', 'Allyson Ross Davies', 'George Kuo', 'Jian Bing Fan', 'Gwyneth Van', 'Paul McGale', 'Ellson Y. Chen', 'Trevor Hastie', 'Pierre R. Villeneuve', 'Thomas Tuschl', 'Richard R. Copley', 'Torben Palshof', 'James R. Hudson', 'Sandra Silberman', 'John McCarthy', 'Wieslaw J. Roth', 'Douglas J. Schwartzentruber', 'Ian Dunham', 'Carol W. Greider', 'Leslie L. Iversen', 'Sean R. Eddy', 'Claudia A. Seipp', 'Frederic M. Richards', 'Lawrence G. Hunsicker', 'Rajinder Kaul', 'Mark R. Chassin', 'Susan E. Folstein', 'Richard Breathnach', 'Thomas R. Dawber', 'E. A. McCulloch', 'Stephen C. Reingold', 'Mukesh Dherani', 'F Taylor', 'Santos A. Susin', 'Simon Myers', 'Ronald H. Reeder', 'Hilary A. Coller', 'James E. Grizzle', 'Lynn A. G. Ries', 'Eugene L. Brown', 'Robert Karasek', 'Edward Witten', 'Myron Flickner', 'Oliver H. Lowry', 'A. L. Hodgkin', 'Kenneth J. Arrow', 'Jorja G. Henikoff', 'G. E. Hutchinson', 'Nancy A. Thornberry', 'Naoufal Zamzami', 'A. P. Alivisatos', 'Vincent P. Stanton', 'E Thomson', 'Deanna M. Church', 'Mark S. Chee', 'Joe M. McCord', 'François Artiguenave', 'Liisa Holm', 'Eladio Viñuela', 'Mary Burden', 'Granger G. Sutton', 'Oliver Smithies', 'Herbert W. Boyer', 'William H. Habig', 'J Graham', 'J. R. Vane', 'Therese Sørlie', 'Audrey Duncanson', 'Marjorie B. Lees', 'Franklin Kim', 'Maurizio Belfiglio', 'C Davies', 'Edmund A. Gehan', 'Jill P. Mesirov', 'Alastair Kent', 'John R. Glover', 'Susan M. Hailpern', 'Patrick Therasse', 'Sarah Hunt', 'Arthur L. Delcher', 'Gavin Sherlock', 'Martin Gibson', 'Robert P. Abelson', 'Kristin Ardlie', 'Timothy B. Stockwell', 'Zhengdong D. Zhang', 'Stewart Craig', 'Charles Dauguet', 'Thomas Jenuwein', 'Dean Burk', 'Joseph Ragaz', 'Renaud Capdeville', 'Halsted R. Holman', 'Mark H. Skolnick', 'Harold Swerdlow', 'Jennifer Baldwin', 'Roger E. Kirk', 'Erik L. L. Sonnhammer', 'Adam Auton', 'Abraham Kagan', 'Marianne Sullivan', 'Torsten N. Wiesel', 'Vishwanath R. Iyer', 'W. Brian Arthur', 'Michael L. Watson', 'Darrel A. Regier', 'Graeme A. Reid', 'D. H. Auston', 'Srinivasa M. Srinivasula', 'Daniel I. Arnon', 'J Clarke', 'Marshal F. Folstein', 'Tullia Lindsten', 'Robert J. Lipshutz', 'Lucy Matthews', 'Edwin M. Southern', 'William B. Kannel', 'James C. Mullikin', 'Matthew Stephens', 'Jane Grimwood', 'Ross L. Prentice', 'Laura Elnitski', 'Alexandre Montpetit', 'M. G. Bawendi', 'Nathan O. Kaplan', 'Jim Gray', 'Elizabeth Lacy', 'Mark Yandell', 'Christopher B. Burge', 'Andrew Kasarskis', 'Alan T. Bankier', 'Clifford L. Johnson', 'Koichiro Tamura', 'Harold Weintraub', 'L. Di Martino', 'Robert A. Weinberg', 'Susan L. Naylor', 'Andrew Wakeham', 'Karin Eiglmeier', 'John G. Kirkwood', 'Scott Uknes', 'Steven E. Scherer', 'Jacques Ferlay', 'David L. Donoho', 'Timothy Fennell', 'Genji Matsuda', 'Barbara E. Stranger', 'Eric A. Jaffe', 'Daphne W. Bell', 'D. Williams Parsons', 'John V. Bennett', 'Matt van de Rijn', 'Parkhurst A. Shore', 'Paul Liska', 'Aram V. Chobanian', 'James J. Heckman', 'Simon L. Howell', 'R. W. Butcher', 'S. M. Zahurak', 'Pamela Whittaker', 'Robert A. Rosati', 'Sarah Nutland', 'David Torrents', 'Yoav Freund', 'Gabor T. Marth', 'Catherine A. Ball', 'T. N. Bhat', 'Mark J. Daly', 'Eric A. Swanson', 'Scott L. Weinrich', 'Robert Katzman', 'Taylor A. Sohn', 'Erding Hu', 'Andreas Thess', 'Sj Wilson', 'Ants Leetmaa', 'P. F. Scholander', 'Allen Newell', 'Argiris Efstratiadis', 'John P. Feighner', 'Lisa French', 'Larry Rubinstein', 'A. L. Page', 'Catherine Michaud', 'Jean Endicott', 'Jacques Monod', 'Anne Dornhorst', 'Kara Dolinski', 'Stanley L. Wallace', 'Kurt J. Greenlund', 'Emily Bernstein', 'Francine Brière', 'Ronald Melzack', 'Gerald M. Rubin', 'Richard Helm', 'C Fox', 'Robert G. Parr', 'David B. Johnson', 'Lon R. Cardon', 'Gail R. Martin', 'David L. Swofford', 'Roland Heilig', 'Helen Goodell', 'John H. Holland', 'Francis M. Miezin', 'Frederick Mosteller', 'J.Ward Kennedy', 'Sidney J. Cutler', 'Michael S. Gottlieb', 'John R. Marler', 'Keith J. Worsley', 'Waldo E. Cohn', 'Joan Keesey', 'Andrew Y. Ng', 'Brian Haynes', 'U. Veronesi', 'Warren J. Hehre', 'Allan Van Oosterom', 'Martin Fishbein', 'Arian Smit', 'James B. Wyngaarden', 'Robin Humphry-Baker', 'David G. Lowe', 'Charles D. Keeling', 'Alan D. Lopez', 'Kimmen Sjölander', 'David G. Kleinbaum', 'W. James Kent', 'J Martin Bland', 'Robert J. Dodson', 'K. S. Novoselov', 'Mercedes Camalli', 'Alexej Abyzov', 'Melvin P. Judkins', 'William H. Majoros', 'D. Lawrence Wickerham', 'Paul Doty', 'Ewan Birney', 'Daniel W. Jones', 'Kung-Yee Liang', 'Gordon Saksena', 'Solange Chamaret', 'Frederick C. Neidhardt', 'John Nash', 'Michael S. Waterman', 'Thomas N. Sato', 'M. Castiglione', 'Geoffrey E. Hinton', 'Wolfgang Kabsch', 'David Julius', 'David L. Pearle', 'Dennis Y. Loh', 'Ronald A. Laskey', 'Thomas C. Chalmers', 'Noriko Sakaguchi', 'Eric Pelletier', 'Jeffrey A. Cutler', 'Thomas J. Palker', 'Michael D. McLellan', 'M. G. Sarngadharan', 'Michael Feolo', 'Taylor Ja', 'Valerie A. Fadok', 'Andreas Rump', 'F. C. Greenwood', 'Vladimir Korinek', 'Reto Ruedy', 'Lore Zech', 'A. C. T. North', 'Peter W. J. Rigby', 'Donald P. Nierlich', 'Edgardo Lander', 'Jeremy D. Glasner', 'Sarah S. Murray', 'Rebecca DerSimonian', 'Clifton F. Mountain', 'David R. Bentley', 'R. A. Fisher', 'Andrew Fire', 'Gordon L. Shulman', 'Anita L. Stewart', 'Ahmedin Jemal', 'Desmond G. Higgins', 'John M. Darley', 'H. J. Stewart', 'Isaac L. Chuang', 'David J. Dooling', 'David W. Tank', 'Yuval Eshed', 'Monica Williams', 'Kikuko Hotta', 'Nj Bell', 'Kazuto Kato', 'Linus Pauling', 'Freddie Bray', 'Bert Vogelstein', 'K McPherson', 'Werner Risau', 'Evan Mauceli', 'Federica Sallusto', 'Charles Atkinson', 'Paul Emsley', 'D. C. Van Essen', 'P Park', 'John D. Lafferty', 'Donna F. Stroup', 'Mark Weiser', 'Charles H. Hennekens', 'Beverley Shields', 'Hugh Paterson', 'Ken Dewar', 'John E. Helzer', 'Sanford L. Palay', 'George Paxinos', 'Liesi E. Hebert', 'Robert C. Onofrio', 'Rachel M. Freathy', 'David A. Hungerford', 'Yasunori Ogura', 'Bruce N. Ames', 'Sam Griffiths-Jones', 'Catrina Fronick', 'Ivan Lieberburg', 'Stephen T. Sherry', 'Charmaine D. Royal', 'N. F. Mott', 'Douglas T. Ross', 'H. Murayama', 'Marshall N. Rosenbluth', 'Rodney M. Hewick', 'W. M. Cowan', 'Karen Hsiao', 'Timo T. Valle', 'Yasushi Totoki', 'Amos Marc Bairoch', 'William A. Knaus', 'William J. Fulkerson', 'Christopher K. Raymond', 'Ross A. Lippert', 'John A. Pople', 'W. Richard McCombie', 'Lora Lewis', 'Michael W. Hunkapiller', 'B. Gutenberg', 'Karen Eilbeck', 'John R. Carson', 'Jeremy White', 'Shelley E. Taylor', 'Tarjei S. Mikkelsen', 'Vincent Ferretti', 'Thomas H. Holmes', 'Karsten Hokamp', 'David Haussler', 'Graham R. Bignell', 'W. Dansgaard', 'Paul Ekman', 'Amy A. Caudy', 'Ingrid Daubechies', 'B. A. Roe', 'A. Howie', 'Joachim Messing', 'Franz Josef Och', 'Eli Robins', 'Bates Dw', 'H. R. Horvitz', 'Michael Ashburner', 'Hamilton O. Smith', 'William N. Anderson', 'Kenneth W. Kinzler', 'Joseph H. Connell', 'Tom Maniatis', 'John M.C. Gutteridge', 'Suzanna E. Lewis', 'Rolf Apweiler', 'Robert I. Levy', 'Paul J. Deutsch', 'Eric W. Wong', 'Robin Milner', 'Keith R. Porter', 'Stefan Taudien', 'S Takashima', 'Michael J. Thun', 'Joe Sambrook', 'E. A. Clarke', 'David W. Leung', 'Marco Punta', 'M. Karplus', 'B. F. Skinner', 'Vishva M. Dixit', 'Andrew G. Fraser', 'Brian R. Lawn', 'Max H. Myers', 'Sewall Wright', 'Martine Van Glabbeke', 'Bruce Culleton', 'Martin L. Hooper', 'Nancy F. Hansen', 'Albert J. Belanger', 'Kirk R. Thomas', 'Andrew Menzies', 'Stanford Moore', 'Miller Ja', 'Max Born', 'Ulrich K. Laemmli', 'Walter Gilbert', 'Tracy M. Bryan', 'G. Flandrin', 'Alfonse T. Masi', 'Giorgio Cocconi', 'James H. Stock', 'Nathan Mantel', 'Kenneth P. Burnham', 'Lawrence E. Shulman', 'Sheila Seal', 'David S. Johnson', 'Gonçalo R. Abecasis', 'Donald E. Groom', 'Ramzi S. Cotran', 'S Clarke', 'Owen P. Hamill', 'A. K. Geim', 'William P. Castelli', 'Jinghui Zhang', 'Folke Skoog', 'Alejandro A. Schäffer', 'George E. Palade', 'Paul E. Meehl', 'Lee Murphy', 'Antman Em', 'René O. Mirimanoff', 'Bonnee Rubinfeld', 'Jennifer J. Anderson', 'Gitanjali M. Singh', 'Anthony J. Cox', 'Richard O. Russell', 'David L. Sackett', 'Felsenfeld Al', 'Jane Peterson', 'H. Craig', 'M Schumacher', 'Sergey V. Morozov', 'Jacqueline Kosecoff', 'P. J. Flory', 'Irving Langmuir', 'Albert H. Coons', 'Roel P. Funke', 'Steven L. Salzberg', 'Alane Gray', 'James W. Curran', 'Lars Hedin', 'G. A. R. Johnston', 'G. Bonadonna', 'Auke Tellegen', 'Michael R. Brent', 'al. et', 'Albert-László Barabási', 'Earl W. Sutherland', 'William Haenszel', 'Dmitriy A. Dikin', 'J. P. Grime', 'N. Ferrara', 'A. D. Bangham', 'Karen A. Ketchum', 'Sidney C. Smith', 'Douglas P. Wagner', 'Don W. Fawcett', 'Steven H. Strogatz', 'Jonathan K. Pritchard', 'Toby Bloom', 'Deborah R. Shapiro', 'Phyllis A. Wingo', 'Ronald L. Rivest', 'I. V. Grigorieva', 'Mary Osborn', 'Glenn Tesler', 'P. J. Sellers', 'F. Sanger', 'David H. Gelfand', 'Howard A. Eder', 'J. L. Weiland', 'Jennifer R. Wortman', 'Daniel McGee', 'G. D. Farquhar', 'John F. Heidelberg', 'Yuen Shing', 'Brenda K. Edwards', 'John Bowlby', 'Ruslan Medzhitov', 'Ching W. Tang', 'Norman A. Doggett', 'Gerald R. Fink', 'Trevor Hawkins', 'Samuel B. Guze', 'Richard R. Hudson', 'Solomon A. Berson', 'Eugene R. Passamani', 'David B. Jaffe', 'Roger Y. Tsien', 'Bigger Jt', 'Victor R. Ambros', 'David H. Culver', 'Donald D. Van Slyke', 'John E. Wennberg', 'McGuire Wl', 'Wayne J. Higgins', 'Jingtao Sun', 'Reginald A. Clift', 'Jitendra Malik', 'Gordon G. Wong', 'Fred Ramsdell', 'Heather Schmidt', 'J. S. Bell', 'P. G. Smith', 'Michael L. Frigge', 'Gilbert Chu', 'Irene M. Stratton', 'Martin Ringwald', 'Katherine W. Timothy', 'William H. Dietz', 'Joseph L. Fleiss', 'E Howard', 'Yuri Lazebnik', 'Cyrus Levinthal', 'Stephen Frank Schaffner', 'M. A. Fiatarone', 'Cole Trapnell', 'J. Milburn Jessup', 'Chris C A Spencer', 'Richard M. Karp', 'Henry A. Erlich', 'Jörg Schultz', 'Gary Struhl', 'Steven R. Gill', 'Patrick Minx', 'Craig H. Mermel', 'L. J. Ignarro', 'Kevin McKernan', 'Ahmed M. El-Nahas', 'Joseph O. Hirschfelder', 'C. Pennypacker', 'Aaron L. Halpern', 'Pere Puigserver', 'James G. Hirsch', 'J. S. Binkley', 'William R. McCabe', 'Andrew H. Wyllie', 'Morton Kramer', 'Alexander Parkhomenko', 'Kenneth Macleod', 'Cyrus Chothia', 'A. H.-J. Wang', 'Helmut Blöcker', 'Tong Ihn Lee', 'Hans-Gerd Lipinski', 'Mary Jane Ferraro', 'Michael Zuker', 'D. C. Wiley', 'Marc Zabeau', 'Jeremy Peterson', 'Henderson Ic', 'Insa Gathmann', 'Patrice J. Morin', 'D. Easton', 'F. Barre-Sinoussi', 'Rodney A. Brooks', 'P.-G. De Gennes', 'Jean Morissette', 'David J. Lockhart', 'Betty J. Flehinger', 'Robert H. Waterston', 'Karl A. Piez', 'Morten Kjeldgaard', 'C Rose', 'Michael E. Greenberg', 'Kazuhiko Kawasaki', 'Marius Wernig', 'Margherita Maffei', 'William J. Martone', 'Kiran Garimella', 'A. Naja', 'Jane G. Schaller', 'Ralph S. Paffenbarger', 'Masatoshi Nei', 'Randall K. Saiki', 'Michael Polanyi', 'Carl R. Woese', 'Nicola Mulder', 'Ilya Spigland', 'Stephen F. Jencks', 'Michael C. Wendl', 'Robert L. Spitzer', 'Marcy Silver', 'Marcus E. Raichle', 'George Scatchard', 'Leonid Kruglyak', 'Lynn R. Sykes', 'Ali Zandieh', 'Kristian Cibulskis', 'Peter Philippsen', 'Joseph Sambrook', 'Robert Kamen', 'James I. Cleeman', 'Hiroaki Hemmi', 'Lucinda Fulton', 'Carole A. Cull', 'David W. Hosmer', 'P. Andrew Futreal', 'Edgar F. Meyer', 'T. J. Sejnowski', 'Paul W. Gast', 'John C. Khoo', 'Lee N. Robins', 'Webb Miller', 'Rory Collins', 'Chad Nusbaum', 'Lynn M. Schriml', 'Martin Poenie', 'Joseph Berkson', 'Eric R. Fearon', 'Raymond D. Miller', 'Sidney P. Colowick', 'Jay D. Pearson', 'Marie-Angèle Morel']
WITH a, COUNT(q) AS num_top_collabs
WHERE num_top_collabs > 0

MATCH (a)-[:AUTHORED]->(q:Quanta)
WHERE a.last_author_yr > 2013 AND q.venue in ['Angewandte Chemie','Blood','Cancer Cell','Cancer Discovery','Cancer Research','Cell','Cell Host & Microbe','Cell Metabolism','Cell Stem Cell','Chemistry & Biology','The EMBO Journal','Genes & Development','Immunity','Journal of Neurology','Journal of the American Chemical Society','JAMA','Journal of Biological Chemistry','Journal of Cell Biology','Journal of Clinical Investigation','Journal of Experimental Medicine','Journal of Medicinal Chemistry','The Lancet','Nature Cell Biology','Nature Chemical Biology','Nature Chemistry','Nature Medicine','Nature Methods','Nature','Nature Biotechnology','The New England Journal of Medicine','Neuron','Nature Genetics','Nature Immunology','Nature Neuroscience','Nature Structural & Molecular Biology','PLOS Biology','PLOS Genetics','PLOS Pathogens','Proceedings of the National Academy of Sciences of the United States of America','Science Signaling','Science Translational Medicine','Science']
    AND a.last_author_yr <= q.year 

WITH a, COUNT(q) AS num_pubs, 
    SUM(q.pageRank_2018) AS tot_rank, 
    MIN(q.pageRank_2018) AS min_rank, 
    MAX(q.pageRank_2018) AS max_rank, 
    percentileCont(q.pageRank_2018, 0.5) AS median_rank 

RETURN a.name AS name, 
    toFloat(tot_rank)/num_pubs AS avg_PR, 
    min_rank AS min_PR, 
    max_rank AS max_PR, 
    median_rank AS median_PR"
    , "/data/csv/PR_Early_Profs_3.csv", {batchSize:100, iterateList:true, parallel:true})

"""
df_pr = query_to_df(query, graph)

query = """
call apoc.export.csv.query(
"MATCH (a:Top42Author)
WHERE a.last_author_yr > 2013 

MATCH (a)-[:AUTHORED]->(q:Quanta)<-[:AUTHORED]-(b:Author)
WHERE q.venue in ['Angewandte Chemie','Blood','Cancer Cell','Cancer Discovery','Cancer Research','Cell','Cell Host & Microbe','Cell Metabolism','Cell Stem Cell','Chemistry & Biology','The EMBO Journal','Genes & Development','Immunity','Journal of Neurology','Journal of the American Chemical Society','JAMA','Journal of Biological Chemistry','Journal of Cell Biology','Journal of Clinical Investigation','Journal of Experimental Medicine','Journal of Medicinal Chemistry','The Lancet','Nature Cell Biology','Nature Chemical Biology','Nature Chemistry','Nature Medicine','Nature Methods','Nature','Nature Biotechnology','The New England Journal of Medicine','Neuron','Nature Genetics','Nature Immunology','Nature Neuroscience','Nature Structural & Molecular Biology','PLOS Biology','PLOS Genetics','PLOS Pathogens','Proceedings of the National Academy of Sciences of the United States of America','Science Signaling','Science Translational Medicine','Science']
    AND q.year < a.last_author_yr
    AND b.name in ['David P. Reed', 'Mikulas Popovic', 'Katherine M. Flegal', 'Boris Reva', 'Emily S. Winn-Deen', 'B. G. Barrell', 'Richard G. Margolese', 'John M. Chirgwin', 'Floyd Ratliff', 'Joan Massagué', 'Sarah Milne', 'M. Overgaard', 'Joseph L. Goldstein', 'Todd M. Lowe', 'Jonathan Marchini', 'Peter J. Ramadge', 'Daniel R. Marshak', 'Stephen M. J. Searle', 'Leon Festinger', 'James Kennedy', 'Michael B. Mock', 'Juha Kononen', 'Julia B. Lewis', 'Jonathan R. Edwards', 'Victor E. Velculescu', 'Peter J. Turnbaugh', 'Akira Nagafuchi', 'Amy E. Pasquinelli', 'Norman Arnheim', 'Thomas D. Schmittgen', 'Linda M. Rolnitzky', 'Rodrigo Lopez', 'J.-C. Gazet', 'Joel S. Hayflick', 'P. C. Eklund', 'Stefanie S. Jeffrey', 'Camille Parmesan', 'N. Cabrera', 'Andrew J. Mungall', 'Robert Baertsch', 'Robert E. Bruccoleri', 'Robert S. Pinals', 'Helen Skaletsky', 'Keith Nelson', 'J. Richard Hackman', 'J. Michael Cherry', 'Mark F. Pittenger', 'Richa Agarwala', 'Mark A. DePristo', 'James H. Ware', 'Serafim Batzoglou', 'Lee A. Green', 'J. E. Till', 'Elizabeth Ward', 'Sheila Scully', 'William T. Friedewald', 'Allan M. Skene', 'V. Narry Kim', 'Tina Graves', 'Elaine R. Mardis', 'George W. Torrance', 'Robert L. Coffman', 'Andreas Heger', 'Ellen E. Walters', 'Jeremy D. Selengut', 'Ruth E. Ley', 'C. V. Shank', 'Gregory J. Hannon', 'Donald S. Rae', 'James A. Stockman', 'Saira Salahuddin', 'J. G. Bednorz', 'D. von Fournier', 'J. F. A. P. Miller', 'Scott Schwartz', 'Ivica Letunic', 'Heike Fiegler', 'J. Gillis', 'Russell Ross', 'Christie Kovar', 'Bruce W. Birren', 'Richard S. Lazarus', 'Helga Boedtker', 'George A. Miller', 'George H. Mead', 'Arthur Purdy Stout', 'LaDeana W. Hillier', 'William A. Sibley', 'Michael P. Bevilacqua', 'Michele L. Pearson', 'Jean-Roger Le Gall', 'M. R. Cooper', 'Jean Thierry-Mieg', 'Joanna E. Siegel', 'Raffaella Sordella', 'David J. Spiegelhalter', 'Frans Bruyninckx', 'Bruce M. Spiegelman', 'George G. J. M. Kuiper', 'Stacey Gabriel', 'Seymour S. Kety', 'Kenneth F. Schulz', 'S. L. McCall', 'Morris W. Foster', 'Huaiyu Mi', 'Nick Patterson', 'Donna K. Slonim', 'Richard H. Chapman', 'Clare Seamark', 'Saul A. Teukolsky', 'Craig C. Mello', 'E Robinson', 'A E Weyman', 'Hartmut Land', 'Ramon Parsons', 'Katsuaki Hoshino', 'George J. Taylor', 'Deborah Wentworth', 'M Richard', 'Thomas E. Carew', 'David J. Lipman', 'Erich Bluhmki', 'John D. McPherson', 'Robert J. Garrison', 'William F. Novotny', 'Kary B. Mullis', 'Robert M. Weisskoff', 'Zemin Ning', 'Daniel Shoemaker', 'M Bond', 'Jillian Boreham', 'Zoltán N. Oltvai', 'Jay R. Ritter', 'Leland H. Hartwell', 'G. Binnig', 'Tomaso Poggio', 'Nathan Maccoby', 'Donald D. Newmeyer', 'Benjamin A. Barnes', 'Charles Tanford', 'M. Michele Manos', 'Otto A. Bessey', 'Yudong D. He', 'Herbert A. Simon', 'Deborah Estrin', 'John V. Moran', 'Gabor Gyapay', 'Michael Shi', 'Edward J. Roccella', 'Richard D. Piner', 'Taro Kawai', 'Erica Sodergren', 'Stephen J. Aldington', 'M. Y. Kim', 'Gilean McVean', 'John P. Perdew', 'Scott Shenker', 'S. Moncada', 'Anders Krogh', 'Jerome Cornfield', 'Ralph M. Steinman', 'M. E. J. Newman', 'Jeffrey C. Lee', 'Benjamin D. Hall', 'Miriam J. Alter', 'Sidney Udenfriend', 'James K. V. Willson', 'Mark S. Guyer', 'Julian Peto', 'James E. Galagan', 'Alice C. Young', 'Theophil Staehelin', 'Steven D. Averbuch', 'Edmund G. Lowrie', 'Cliff Tabin', 'P Sandercock', 'Nathan Goodman', 'Richard A. Shafer', 'Steve Rozen', 'A. R. Hunt', 'J.A. van Dongen', 'Mary C. Betlach', 'Michael C. Zody', 'Harry Towbin', 'Myunghee C. Paik', 'Mark Diekhans', 'Michael Gent', 'Mark S. Boguski', 'Hongjie Dai', 'David V. Goeddel', 'Andrew Neil', 'Yunzhen Cao', 'M. H. Gault', 'Paul Meier', 'Robert E. Kleiger', 'Kim Rutherford', 'William R. Pearson', 'Janan T. Eppig', 'Bert Sakmann', 'Stuart McLaren', 'R Parker', 'James H. Jandl', 'David P. Bartel', 'C Redmond', 'Jerome Vinograd', 'Mark Dickson', 'Barry A. Morgan', 'Julie D. Thompson', 'Mike Rothe', 'Tobias Doerks', 'Jeffrey G. Reid', 'D. J. E. Ingram', 'Yuling Hong', 'Barbara J. McNeil', 'Robert Tibshirani', 'Jeremy H. Burroughes', 'Ueli Gubler', 'C. Milstein', 'Paul J. Flory', 'A. K. Mitchell', 'Mark T. Ross', 'I. M. Stratton', 'P. D. Home', 'Andrew P. Holmes', 'C. Arden Pope', 'Tim R. Mosmann', 'Richard Peto', 'Richard C. Graham', 'John M. Leonard', 'Arthur B. Lamb', 'Benoit B. Mandelbrot', 'David E. Goldberg', 'E. Kellenberger', 'F. Schedin', 'C. K. Osborne', 'Nicholas Kassebaum', 'Ch. Gerber', 'Russell Turner', 'Stéphane Guindon', 'Frank E. Speizer', 'Susan G. Arbuck', 'Thomas D. Willis', 'R W Davis', 'Phillip D. Zamore', 'Nan M. Laird', 'Barth Hoogstraten', 'Kim C. Worley', 'Hugo Muench', 'Douglas Hanahan', 'Margaret D. Carroll', 'Gary Ruvkun', 'B. E. Warren', 'Catherine A. Lozupone', 'Robert F. Meenan', 'Naruya Saitou', 'James S. Marks', 'Wilson Ia', 'Minze Stuiver', 'Seymour Benzer', 'Terrence S. Furey', 'Judah Folkman', 'Richard E. Nisbett', 'Pui Yan Kwok', 'Mark A. Saper', 'Erika Schmid', 'Marita Cross', 'Dennis G. Ballinger', 'Elisabeth Burdick', 'D. Wayne Taylor', 'Michele Cargill', 'Fred Ederer', 'Loren F. Hiratzka', 'Oliver Schwardt', 'Saul Genuth', 'Stuart Bunting', 'Arnold Oliphant', 'N. E. Day', 'Paul Predki', 'Paul D. Sorlie', 'Donna M. Rennick', 'Louis A. Tartaglia', 'Parnesh Raniga', 'Robert Axelrod', 'A. McGuire', 'Stewart Scherer', 'Lucian L. Leape', 'U. J. Lewis', 'J Burns', 'Sarah E. Fowler', 'Bernard B. Brodie', 'Ghia Euskirchen', 'Kate Downes', 'Scott M. Hammond', 'Robert D. Fleischmann', 'Robert M. Kuhn', 'Richard J. Brand', 'Jerome Y. Lettvin', 'Thomas R. Fleming', 'Joseph D. Mosca', 'Erin Pleasance', 'Nahida H. Gordon', 'Calvin B. Harley', 'Russell Higuchi', 'Réka Albert', 'Keaven M. Anderson', 'J. F. Forbes', 'M Fernandez', 'Zhiping Gu', 'Bala Swaminathan', 'Jeffrey Vieira', 'Annie Yang', 'P. B. Hirsch', 'R. S. J. Frackowiak', 'Tavia Gordon', 'Ralf W. Grosse-Kunstleve', 'James R. Rice', 'V. S. Kalyanaraman', 'Daniel C. Koboldt', 'James B. Hicks', 'Jerome S. Bruner', 'Elia Stupka', 'Michael C. Jensen', 'Randi Y. Leavitt', 'Stephen F. Altschul', 'Hidemi Watanabe', 'Andrey Sivachenko', 'William J. Rutter', 'Michael Fiorino', 'Louis Fehrenbacher', 'James J. Gibson', 'Béatrice Vanbervliet', 'Richard C. Eastman', 'Qui-Lim Choo', 'Rodger Staden', 'Jeffrey C. Barrett', 'Donald B. Rubin', 'Fredrik Ronquist', 'Fuli Yu', 'Reubin Andres', 'Jacob Cohen', 'Joseph H. Roe', 'Troy Moore', 'David Tilman', 'Zhongwu Lai', 'Jonathan Sebat', 'Wonhee Jang', 'David W. Bilheimer', 'P. Armitage', 'Kalevi Pyörälä', 'John E. Ware', 'J. J. Hopfield', 'Norman E. Breslow', 'Julius Marmur', 'J. D. A. Van Embden', 'Christoph Kalka', 'Eric S. Lander', 'George L. Gabor Miklos', 'Douglas B. Rusch', 'Katherine S. Elliott', 'Alan Coulson', 'Jeffrey N. Rottman', 'Scott V. Tingey', 'Arne Tiselius', 'Dennis A. Gilbert', 'Wendy Winckler', 'Sean Humphray', 'Max Halperin', 'E. Wolf', 'Douglas L. Johnson', 'R. C. Turner', 'Arthur M. Sobol', 'Cladd E. Stevens', 'S. Cobbe', 'E. P. Greenberg', 'Howard H. Hiatt', 'P. J. Randle', 'E. Orowan', 'Aaron T. Beck', 'M Klagsbrun', 'Yves-Alain Barde', 'Daniel Kahneman', 'Robert T. DeBoy', 'G. E. Palade', 'Adam Frankish', 'Sam T. Roweis', 'Andersen Kw', 'Thomas J. Dull', 'Michaele C. Christian', 'David R. Cox', 'Donald Geman', 'M. H. F. Wilkins', 'William J. Henzel', 'Brygg Ullmer', 'Carol Redmond', 'Michael B. Eisen', 'Chang Gong Liu', 'Paul Milgrom', 'Trampisch Hj', 'M. D. Poulik', 'Daniel W. Bradley', 'Carl-Magnus Rudenstam', 'Albert László Barabási', 'David S. Hogness', 'Michael L. Metzker', 'Donald Pearson', 'J. S. Turner', 'James F. Fries', 'Suzannah Bumpstead', 'Ben Z. Locke', 'Marianne Kearney', 'R. Mead', 'Tim Hubbard', 'William Saurin', 'R. R. Ernst', 'Federico Girosi', 'Peter J. Sabo', 'Alex Bateman', 'Dienstag Jl', 'Ed Harlow', 'Carrie Sougnez', 'Guy Plunkett', 'Richard Bellman', 'Aaron McKenna', 'Gary J. Olsen', 'Lawrence L. Kupper', 'Mark D. Adams', 'Luther T. Clark', 'Janet M. Corrigan', 'A. F. Huxley', 'Nigel P. Carter', 'D Wright', 'William H. Stein', 'Robert S. Fulton', 'Heidi Greulich', 'Sankar Swaminathan', 'Toby J. Gibson', 'Simon G. Gregory', 'Aravind Subramanian', 'C. Katz', 'Tetsuo Maita', 'Helena Hämäläinen', 'Eugene W. Myers', 'Rhian Gwilliam', 'Anup Madan', 'Forrest E. Kendall', 'Cynthia L. Ogden', 'Aileen Robertson', 'Harold T. Dodge', 'Marilyn Bergner', 'D. R. Cox', 'G. I. Taylor', 'Jonathan Butler', 'Andrew B. Lassar', 'S Mason', 'Roland M. H. Schein', 'Margo A. Denke', 'Richard M. Goldstein', 'Robert B. Catalano', 'Alexander E. Urban', 'Harvey J. Alter', 'Monique G. Lê', 'Gyorgy Hutvagner', 'Darren A. Natale', 'Stanley J. Korsmeyer', 'J. Craig Venter', 'D Nelson', 'Richard Tizard', 'W Jack', 'Lee Rowen', 'Hailing Hsu', 'Robert P. Gaynes', 'Donald R. Huffman', 'Richard H. Creech', 'Michael H. Kutner', 'Michael A. Saunders', 'R Peto', 'Carol J. Bult', 'Ian Korf', 'D. R. Curtis', 'Michael Held', 'Michael R. Hee', 'Ali A. Mokdad', 'Frank Delaglio', 'Donald S. Fredrickson', 'Jeffrey S. Beck', 'Shirley Zhu', 'Jorge J. Yunis', 'Richard A. Olshen', 'David Altshuler', 'Gabriele Bergers', 'Sandra W. Clifton', 'Colin N. Dewey', 'Georges Köhler', 'Paul T. Spellman', 'Stanley Lemeshow', 'Dominic A. Scudiero', 'Basil M. Rifkind', 'Walter M. Fitch', 'Endel Tulving', 'J. M. Bland', 'Nikola P. Pavletich', 'Nawab Qizilbash', 'Raymond J. MacDonald', 'W. H. Zachariasen', 'Daniel T. Colbert', 'D. R. Matthews', 'Patrick Brill-Edwards', 'Michael K. Showe', 'Marc van de Wetering', 'Bela Julesz', 'Earl Hubbell', 'Scott L. Zeger', 'Peter K. Wellauer', 'Zbyszek Otwinowski', 'Charles T. Kresge', 'Helen Van Vunakis', 'Nicoletta Sacchi', 'Douglas G. Altman', 'R Doll', 'Richard W. Reynolds', 'A. Klug', 'Amos Tversky', 'Quinn L. Deveraux', 'Patrick O. Brown', 'Patrick D. Wall', 'David Botstein', 'K Matsushima', 'Brufman G', 'Zhaodan Cao', 'Elisabeth Buchdunger', 'Alex Pentland', 'F. P. Bowden', 'J Holmes', 'Kenneth J. Livak', 'K Sakai', 'Hoda Khouri', 'Jerzy Neyman', 'R. Willox', 'Raymond G. Goodwin', 'Richard Durbin', 'Alan J. Buckler', 'Fu Lu', 'Gordon J', 'Pablo Tamayo', 'Shimon Ullman', 'Gary W. Trucks', 'Morris J. Karnovsky', 'J. S. Waugh', 'Wen Lin Kuo', 'Nicholas B. Lydon', 'Amy J. Weiner', 'Anne S. Olsen', 'Allan C. Wilson', 'Daniel H. Haft', 'Stephen E. Haynesworth', 'Marek Kubin', 'Carol Churcher', 'Todd D. Taylor', 'Jane Rogers', 'Teresa C. Horan', 'S. Korzeniowski', 'Nick Goldman', 'John P. Huelsenbeck', 'Huyen Dinh', 'Maynard V. Olson', 'Vern Paxson', 'Joel E. Richardson', 'John C. Cassel', 'Josep F. Abril', 'Robert F. Kibler', 'H. Gervásio', 'David I. Gottlieb', 'Sarah Edkins', 'Howard K. Schachman', 'David Beach', 'Andrew P. Feinberg', 'Hilary King', 'John C. Doyle', 'Jon Teague', 'Laurence Kennedy', 'Anthony R. Kerlavage', 'Drummond Rennie', 'Allyson Ross Davies', 'George Kuo', 'Jian Bing Fan', 'Gwyneth Van', 'Paul McGale', 'Ellson Y. Chen', 'Trevor Hastie', 'Pierre R. Villeneuve', 'Thomas Tuschl', 'Richard R. Copley', 'Torben Palshof', 'James R. Hudson', 'Sandra Silberman', 'John McCarthy', 'Wieslaw J. Roth', 'Douglas J. Schwartzentruber', 'Ian Dunham', 'Carol W. Greider', 'Leslie L. Iversen', 'Sean R. Eddy', 'Claudia A. Seipp', 'Frederic M. Richards', 'Lawrence G. Hunsicker', 'Rajinder Kaul', 'Mark R. Chassin', 'Susan E. Folstein', 'Richard Breathnach', 'Thomas R. Dawber', 'E. A. McCulloch', 'Stephen C. Reingold', 'Mukesh Dherani', 'F Taylor', 'Santos A. Susin', 'Simon Myers', 'Ronald H. Reeder', 'Hilary A. Coller', 'James E. Grizzle', 'Lynn A. G. Ries', 'Eugene L. Brown', 'Robert Karasek', 'Edward Witten', 'Myron Flickner', 'Oliver H. Lowry', 'A. L. Hodgkin', 'Kenneth J. Arrow', 'Jorja G. Henikoff', 'G. E. Hutchinson', 'Nancy A. Thornberry', 'Naoufal Zamzami', 'A. P. Alivisatos', 'Vincent P. Stanton', 'E Thomson', 'Deanna M. Church', 'Mark S. Chee', 'Joe M. McCord', 'François Artiguenave', 'Liisa Holm', 'Eladio Viñuela', 'Mary Burden', 'Granger G. Sutton', 'Oliver Smithies', 'Herbert W. Boyer', 'William H. Habig', 'J Graham', 'J. R. Vane', 'Therese Sørlie', 'Audrey Duncanson', 'Marjorie B. Lees', 'Franklin Kim', 'Maurizio Belfiglio', 'C Davies', 'Edmund A. Gehan', 'Jill P. Mesirov', 'Alastair Kent', 'John R. Glover', 'Susan M. Hailpern', 'Patrick Therasse', 'Sarah Hunt', 'Arthur L. Delcher', 'Gavin Sherlock', 'Martin Gibson', 'Robert P. Abelson', 'Kristin Ardlie', 'Timothy B. Stockwell', 'Zhengdong D. Zhang', 'Stewart Craig', 'Charles Dauguet', 'Thomas Jenuwein', 'Dean Burk', 'Joseph Ragaz', 'Renaud Capdeville', 'Halsted R. Holman', 'Mark H. Skolnick', 'Harold Swerdlow', 'Jennifer Baldwin', 'Roger E. Kirk', 'Erik L. L. Sonnhammer', 'Adam Auton', 'Abraham Kagan', 'Marianne Sullivan', 'Torsten N. Wiesel', 'Vishwanath R. Iyer', 'W. Brian Arthur', 'Michael L. Watson', 'Darrel A. Regier', 'Graeme A. Reid', 'D. H. Auston', 'Srinivasa M. Srinivasula', 'Daniel I. Arnon', 'J Clarke', 'Marshal F. Folstein', 'Tullia Lindsten', 'Robert J. Lipshutz', 'Lucy Matthews', 'Edwin M. Southern', 'William B. Kannel', 'James C. Mullikin', 'Matthew Stephens', 'Jane Grimwood', 'Ross L. Prentice', 'Laura Elnitski', 'Alexandre Montpetit', 'M. G. Bawendi', 'Nathan O. Kaplan', 'Jim Gray', 'Elizabeth Lacy', 'Mark Yandell', 'Christopher B. Burge', 'Andrew Kasarskis', 'Alan T. Bankier', 'Clifford L. Johnson', 'Koichiro Tamura', 'Harold Weintraub', 'L. Di Martino', 'Robert A. Weinberg', 'Susan L. Naylor', 'Andrew Wakeham', 'Karin Eiglmeier', 'John G. Kirkwood', 'Scott Uknes', 'Steven E. Scherer', 'Jacques Ferlay', 'David L. Donoho', 'Timothy Fennell', 'Genji Matsuda', 'Barbara E. Stranger', 'Eric A. Jaffe', 'Daphne W. Bell', 'D. Williams Parsons', 'John V. Bennett', 'Matt van de Rijn', 'Parkhurst A. Shore', 'Paul Liska', 'Aram V. Chobanian', 'James J. Heckman', 'Simon L. Howell', 'R. W. Butcher', 'S. M. Zahurak', 'Pamela Whittaker', 'Robert A. Rosati', 'Sarah Nutland', 'David Torrents', 'Yoav Freund', 'Gabor T. Marth', 'Catherine A. Ball', 'T. N. Bhat', 'Mark J. Daly', 'Eric A. Swanson', 'Scott L. Weinrich', 'Robert Katzman', 'Taylor A. Sohn', 'Erding Hu', 'Andreas Thess', 'Sj Wilson', 'Ants Leetmaa', 'P. F. Scholander', 'Allen Newell', 'Argiris Efstratiadis', 'John P. Feighner', 'Lisa French', 'Larry Rubinstein', 'A. L. Page', 'Catherine Michaud', 'Jean Endicott', 'Jacques Monod', 'Anne Dornhorst', 'Kara Dolinski', 'Stanley L. Wallace', 'Kurt J. Greenlund', 'Emily Bernstein', 'Francine Brière', 'Ronald Melzack', 'Gerald M. Rubin', 'Richard Helm', 'C Fox', 'Robert G. Parr', 'David B. Johnson', 'Lon R. Cardon', 'Gail R. Martin', 'David L. Swofford', 'Roland Heilig', 'Helen Goodell', 'John H. Holland', 'Francis M. Miezin', 'Frederick Mosteller', 'J.Ward Kennedy', 'Sidney J. Cutler', 'Michael S. Gottlieb', 'John R. Marler', 'Keith J. Worsley', 'Waldo E. Cohn', 'Joan Keesey', 'Andrew Y. Ng', 'Brian Haynes', 'U. Veronesi', 'Warren J. Hehre', 'Allan Van Oosterom', 'Martin Fishbein', 'Arian Smit', 'James B. Wyngaarden', 'Robin Humphry-Baker', 'David G. Lowe', 'Charles D. Keeling', 'Alan D. Lopez', 'Kimmen Sjölander', 'David G. Kleinbaum', 'W. James Kent', 'J Martin Bland', 'Robert J. Dodson', 'K. S. Novoselov', 'Mercedes Camalli', 'Alexej Abyzov', 'Melvin P. Judkins', 'William H. Majoros', 'D. Lawrence Wickerham', 'Paul Doty', 'Ewan Birney', 'Daniel W. Jones', 'Kung-Yee Liang', 'Gordon Saksena', 'Solange Chamaret', 'Frederick C. Neidhardt', 'John Nash', 'Michael S. Waterman', 'Thomas N. Sato', 'M. Castiglione', 'Geoffrey E. Hinton', 'Wolfgang Kabsch', 'David Julius', 'David L. Pearle', 'Dennis Y. Loh', 'Ronald A. Laskey', 'Thomas C. Chalmers', 'Noriko Sakaguchi', 'Eric Pelletier', 'Jeffrey A. Cutler', 'Thomas J. Palker', 'Michael D. McLellan', 'M. G. Sarngadharan', 'Michael Feolo', 'Taylor Ja', 'Valerie A. Fadok', 'Andreas Rump', 'F. C. Greenwood', 'Vladimir Korinek', 'Reto Ruedy', 'Lore Zech', 'A. C. T. North', 'Peter W. J. Rigby', 'Donald P. Nierlich', 'Edgardo Lander', 'Jeremy D. Glasner', 'Sarah S. Murray', 'Rebecca DerSimonian', 'Clifton F. Mountain', 'David R. Bentley', 'R. A. Fisher', 'Andrew Fire', 'Gordon L. Shulman', 'Anita L. Stewart', 'Ahmedin Jemal', 'Desmond G. Higgins', 'John M. Darley', 'H. J. Stewart', 'Isaac L. Chuang', 'David J. Dooling', 'David W. Tank', 'Yuval Eshed', 'Monica Williams', 'Kikuko Hotta', 'Nj Bell', 'Kazuto Kato', 'Linus Pauling', 'Freddie Bray', 'Bert Vogelstein', 'K McPherson', 'Werner Risau', 'Evan Mauceli', 'Federica Sallusto', 'Charles Atkinson', 'Paul Emsley', 'D. C. Van Essen', 'P Park', 'John D. Lafferty', 'Donna F. Stroup', 'Mark Weiser', 'Charles H. Hennekens', 'Beverley Shields', 'Hugh Paterson', 'Ken Dewar', 'John E. Helzer', 'Sanford L. Palay', 'George Paxinos', 'Liesi E. Hebert', 'Robert C. Onofrio', 'Rachel M. Freathy', 'David A. Hungerford', 'Yasunori Ogura', 'Bruce N. Ames', 'Sam Griffiths-Jones', 'Catrina Fronick', 'Ivan Lieberburg', 'Stephen T. Sherry', 'Charmaine D. Royal', 'N. F. Mott', 'Douglas T. Ross', 'H. Murayama', 'Marshall N. Rosenbluth', 'Rodney M. Hewick', 'W. M. Cowan', 'Karen Hsiao', 'Timo T. Valle', 'Yasushi Totoki', 'Amos Marc Bairoch', 'William A. Knaus', 'William J. Fulkerson', 'Christopher K. Raymond', 'Ross A. Lippert', 'John A. Pople', 'W. Richard McCombie', 'Lora Lewis', 'Michael W. Hunkapiller', 'B. Gutenberg', 'Karen Eilbeck', 'John R. Carson', 'Jeremy White', 'Shelley E. Taylor', 'Tarjei S. Mikkelsen', 'Vincent Ferretti', 'Thomas H. Holmes', 'Karsten Hokamp', 'David Haussler', 'Graham R. Bignell', 'W. Dansgaard', 'Paul Ekman', 'Amy A. Caudy', 'Ingrid Daubechies', 'B. A. Roe', 'A. Howie', 'Joachim Messing', 'Franz Josef Och', 'Eli Robins', 'Bates Dw', 'H. R. Horvitz', 'Michael Ashburner', 'Hamilton O. Smith', 'William N. Anderson', 'Kenneth W. Kinzler', 'Joseph H. Connell', 'Tom Maniatis', 'John M.C. Gutteridge', 'Suzanna E. Lewis', 'Rolf Apweiler', 'Robert I. Levy', 'Paul J. Deutsch', 'Eric W. Wong', 'Robin Milner', 'Keith R. Porter', 'Stefan Taudien', 'S Takashima', 'Michael J. Thun', 'Joe Sambrook', 'E. A. Clarke', 'David W. Leung', 'Marco Punta', 'M. Karplus', 'B. F. Skinner', 'Vishva M. Dixit', 'Andrew G. Fraser', 'Brian R. Lawn', 'Max H. Myers', 'Sewall Wright', 'Martine Van Glabbeke', 'Bruce Culleton', 'Martin L. Hooper', 'Nancy F. Hansen', 'Albert J. Belanger', 'Kirk R. Thomas', 'Andrew Menzies', 'Stanford Moore', 'Miller Ja', 'Max Born', 'Ulrich K. Laemmli', 'Walter Gilbert', 'Tracy M. Bryan', 'G. Flandrin', 'Alfonse T. Masi', 'Giorgio Cocconi', 'James H. Stock', 'Nathan Mantel', 'Kenneth P. Burnham', 'Lawrence E. Shulman', 'Sheila Seal', 'David S. Johnson', 'Gonçalo R. Abecasis', 'Donald E. Groom', 'Ramzi S. Cotran', 'S Clarke', 'Owen P. Hamill', 'A. K. Geim', 'William P. Castelli', 'Jinghui Zhang', 'Folke Skoog', 'Alejandro A. Schäffer', 'George E. Palade', 'Paul E. Meehl', 'Lee Murphy', 'Antman Em', 'René O. Mirimanoff', 'Bonnee Rubinfeld', 'Jennifer J. Anderson', 'Gitanjali M. Singh', 'Anthony J. Cox', 'Richard O. Russell', 'David L. Sackett', 'Felsenfeld Al', 'Jane Peterson', 'H. Craig', 'M Schumacher', 'Sergey V. Morozov', 'Jacqueline Kosecoff', 'P. J. Flory', 'Irving Langmuir', 'Albert H. Coons', 'Roel P. Funke', 'Steven L. Salzberg', 'Alane Gray', 'James W. Curran', 'Lars Hedin', 'G. A. R. Johnston', 'G. Bonadonna', 'Auke Tellegen', 'Michael R. Brent', 'al. et', 'Albert-László Barabási', 'Earl W. Sutherland', 'William Haenszel', 'Dmitriy A. Dikin', 'J. P. Grime', 'N. Ferrara', 'A. D. Bangham', 'Karen A. Ketchum', 'Sidney C. Smith', 'Douglas P. Wagner', 'Don W. Fawcett', 'Steven H. Strogatz', 'Jonathan K. Pritchard', 'Toby Bloom', 'Deborah R. Shapiro', 'Phyllis A. Wingo', 'Ronald L. Rivest', 'I. V. Grigorieva', 'Mary Osborn', 'Glenn Tesler', 'P. J. Sellers', 'F. Sanger', 'David H. Gelfand', 'Howard A. Eder', 'J. L. Weiland', 'Jennifer R. Wortman', 'Daniel McGee', 'G. D. Farquhar', 'John F. Heidelberg', 'Yuen Shing', 'Brenda K. Edwards', 'John Bowlby', 'Ruslan Medzhitov', 'Ching W. Tang', 'Norman A. Doggett', 'Gerald R. Fink', 'Trevor Hawkins', 'Samuel B. Guze', 'Richard R. Hudson', 'Solomon A. Berson', 'Eugene R. Passamani', 'David B. Jaffe', 'Roger Y. Tsien', 'Bigger Jt', 'Victor R. Ambros', 'David H. Culver', 'Donald D. Van Slyke', 'John E. Wennberg', 'McGuire Wl', 'Wayne J. Higgins', 'Jingtao Sun', 'Reginald A. Clift', 'Jitendra Malik', 'Gordon G. Wong', 'Fred Ramsdell', 'Heather Schmidt', 'J. S. Bell', 'P. G. Smith', 'Michael L. Frigge', 'Gilbert Chu', 'Irene M. Stratton', 'Martin Ringwald', 'Katherine W. Timothy', 'William H. Dietz', 'Joseph L. Fleiss', 'E Howard', 'Yuri Lazebnik', 'Cyrus Levinthal', 'Stephen Frank Schaffner', 'M. A. Fiatarone', 'Cole Trapnell', 'J. Milburn Jessup', 'Chris C A Spencer', 'Richard M. Karp', 'Henry A. Erlich', 'Jörg Schultz', 'Gary Struhl', 'Steven R. Gill', 'Patrick Minx', 'Craig H. Mermel', 'L. J. Ignarro', 'Kevin McKernan', 'Ahmed M. El-Nahas', 'Joseph O. Hirschfelder', 'C. Pennypacker', 'Aaron L. Halpern', 'Pere Puigserver', 'James G. Hirsch', 'J. S. Binkley', 'William R. McCabe', 'Andrew H. Wyllie', 'Morton Kramer', 'Alexander Parkhomenko', 'Kenneth Macleod', 'Cyrus Chothia', 'A. H.-J. Wang', 'Helmut Blöcker', 'Tong Ihn Lee', 'Hans-Gerd Lipinski', 'Mary Jane Ferraro', 'Michael Zuker', 'D. C. Wiley', 'Marc Zabeau', 'Jeremy Peterson', 'Henderson Ic', 'Insa Gathmann', 'Patrice J. Morin', 'D. Easton', 'F. Barre-Sinoussi', 'Rodney A. Brooks', 'P.-G. De Gennes', 'Jean Morissette', 'David J. Lockhart', 'Betty J. Flehinger', 'Robert H. Waterston', 'Karl A. Piez', 'Morten Kjeldgaard', 'C Rose', 'Michael E. Greenberg', 'Kazuhiko Kawasaki', 'Marius Wernig', 'Margherita Maffei', 'William J. Martone', 'Kiran Garimella', 'A. Naja', 'Jane G. Schaller', 'Ralph S. Paffenbarger', 'Masatoshi Nei', 'Randall K. Saiki', 'Michael Polanyi', 'Carl R. Woese', 'Nicola Mulder', 'Ilya Spigland', 'Stephen F. Jencks', 'Michael C. Wendl', 'Robert L. Spitzer', 'Marcy Silver', 'Marcus E. Raichle', 'George Scatchard', 'Leonid Kruglyak', 'Lynn R. Sykes', 'Ali Zandieh', 'Kristian Cibulskis', 'Peter Philippsen', 'Joseph Sambrook', 'Robert Kamen', 'James I. Cleeman', 'Hiroaki Hemmi', 'Lucinda Fulton', 'Carole A. Cull', 'David W. Hosmer', 'P. Andrew Futreal', 'Edgar F. Meyer', 'T. J. Sejnowski', 'Paul W. Gast', 'John C. Khoo', 'Lee N. Robins', 'Webb Miller', 'Rory Collins', 'Chad Nusbaum', 'Lynn M. Schriml', 'Martin Poenie', 'Joseph Berkson', 'Eric R. Fearon', 'Raymond D. Miller', 'Sidney P. Colowick', 'Jay D. Pearson', 'Marie-Angèle Morel']
WITH a, COUNT(q) AS num_top_collabs
WHERE num_top_collabs > 0

MATCH (a)-[:AUTHORED]->(q:Quanta)
WHERE a.last_author_yr > 2013 AND q.venue in ['Angewandte Chemie','Blood','Cancer Cell','Cancer Discovery','Cancer Research','Cell','Cell Host & Microbe','Cell Metabolism','Cell Stem Cell','Chemistry & Biology','The EMBO Journal','Genes & Development','Immunity','Journal of Neurology','Journal of the American Chemical Society','JAMA','Journal of Biological Chemistry','Journal of Cell Biology','Journal of Clinical Investigation','Journal of Experimental Medicine','Journal of Medicinal Chemistry','The Lancet','Nature Cell Biology','Nature Chemical Biology','Nature Chemistry','Nature Medicine','Nature Methods','Nature','Nature Biotechnology','The New England Journal of Medicine','Neuron','Nature Genetics','Nature Immunology','Nature Neuroscience','Nature Structural & Molecular Biology','PLOS Biology','PLOS Genetics','PLOS Pathogens','Proceedings of the National Academy of Sciences of the United States of America','Science Signaling','Science Translational Medicine','Science']
    AND a.last_author_yr <= q.year 

WITH a, COUNT(q) AS num_pubs, 
    SUM(q.articleRank2018) AS tot_rank, 
    MIN(q.articleRank2018) AS min_rank, 
    MAX(q.articleRank2018) AS max_rank, 
    percentileCont(q.articleRank2018, 0.5) AS median_rank 
    
RETURN a.name AS name, 
    toFloat(tot_rank)/num_pubs AS avg_AR, 
    min_rank AS min_AR, 
    max_rank AS max_AR, 
    median_rank AS median_AR"
    , "/data/csv/AR_Early_Profs_3.csv", {batchSize:100, iterateList:true, parallel:true})

"""
df_ar = query_to_df(query, graph)

Starting query... Done (0.11 minutes).
Starting query... Done (0.10 minutes).


In [7]:
# Num affiliations for each author
query = """
call apoc.export.csv.query(
"MATCH (a:Top42Author)-[:AFFILIATED_WITH]->(o:Organization)
RETURN a.name as name, COUNT(o) as orgs",
"/data/csv/orgs.csv", {batchSize:100, iterateList:true, parallel:true})
"""
df = query_to_df(query, graph)

Starting query... Done (8.25 minutes).


In [ ]:
import pandas as pd
df_disrupt = pd.read_csv('~/scaling-science/data/magone/Disrupt_Early_Profs_3.csv', index_col = 'name', encoding = "UTF-8")
df_collab = pd.read_csv('~/scaling-science/data/magone/Collab_Early_Profs_3.csv', index_col = 'name', encoding = "UTF-8")
df_AR = pd.read_csv('~/scaling-science/data/magone/AR_Early_Profs_3.csv', index_col = 'name', encoding = "UTF-8")
df_PR = pd.read_csv('~/scaling-science/data/magone/PR_Early_Profs_3.csv', index_col = 'name', encoding = "UTF-8")
df_orgs = pd.read_csv('~/scaling-science/data/magone/orgs.csv', index_col = 'name', encoding = "UTF-8")


In [30]:
def top_100_to_csv(trimmed_df, cols, name, num = 100, to_csv = False):
    d = dict()
    for col in cols:
        df_notna = trimmed_df[pd.notna(trimmed_df[col])]
        sorted = df_notna[[col]].sort_values(col, ascending = False).head(num)
        names = sorted.index.tolist()
        values = sorted[col].tolist()
        d[col] = pd.Series(names, index=list(range(1, num+1)))
        d[col+'_val'] = pd.Series(values, list(range(1, num+1)))
    df_rank = pd.DataFrame(d)
    if to_csv:
        df_rank.to_csv('~/scaling-science/data/magone/{}_Early_Profs_rank_2.csv'.format(name), index = False, encoding = "UTF-8")
    return df_rank

In [ ]:
df_disrupt_trim = df_disrupt.join(df_orgs, how = 'inner')
print("Total: %d authors" % df_disrupt_trim.shape[0])
df_disrupt_trim = df_disrupt_trim[df_disrupt_trim['num_disrupt'] >= 5]
df_disrupt_trim = df_disrupt_trim[df_disrupt_trim['orgs'] <= 2000]
print("Remaining after trimming: %d authors" % df_disrupt_trim.shape[0])
top_100_to_csv(df_disrupt_trim, df_disrupt.columns, 'Disrupt', to_csv = True)

In [ ]:
df_collab_trim = df_collab.join(df_orgs, how = 'inner')
print("Total: %d authors" % df_collab_trim.shape[0])
df_collab_trim = df_collab_trim[df_collab_trim['orgs'] <= 2000]
print("Remaining after trimming: %d authors" % df_collab_trim.shape[0])
top_100_to_csv(df_collab_trim, df_collab.columns, 'Collab', to_csv = True)

In [ ]:
df_AR_trim = df_AR.join(df_orgs, how = 'inner')
print("Total: %d authors" % df_AR_trim.shape[0])
df_AR_trim = df_AR_trim[df_AR_trim['orgs'] <= 2000]
print("Remaining after trimming: %d authors" % df_AR_trim.shape[0])
top_100_to_csv(df_AR_trim, df_AR.columns, 'AR', to_csv = True)

In [ ]:
df_PR_trim = df_PR.join(df_orgs, how = 'inner')
print("Total: %d authors" % df_PR_trim.shape[0])
df_PR_trim = df_PR_trim[df_PR_trim['orgs'] <= 2000]
print("Remaining after trimming: %d authors" % df_PR_trim.shape[0])
top_100_to_csv(df_PR_trim, df_PR.columns, 'PR', to_csv = True)